# -*- coding: utf-8 -*-
"""
Integrated DQN-Actor-Critic Architecture for Poisoning Attack Detection
Author: Your Name
Date: 2024
"""

"""# Cell 1: Imports and Mathematical Framework

## Mathematical Framework for DQN-Actor-Critic Based Poisoning Detection

1. Feature Space Definition:
   F = {f ∈ ℝᵈ | f is a network flow feature vector}
   where d is the dimension of extracted features

2. Data Distribution:
   P(x) = Normal network traffic distribution
   Q(x) = Poisoned traffic distribution
   KL(P||Q) = Measure of distribution divergence

3. Attack Space:
   A = {a | a is a poisoning attack vector}
   Impact(a) = Σ ||f_original - f_poisoned||₂

4. Detection Functions:
   DQN: Q(s,a) = 𝔼[R + γ max Q(s',a') | s,a]
   Actor: π(a|s) = P(action=a | state=s)
   Critic: V(s) = 𝔼[Σ γᵗR_t | s₀=s]

5. Combined Detection Score:
   D(x) = α·Q(x) + (1-α)·π(x) + β·KL(P||Q_x)
   where Q_x is the estimated distribution at point x
"""

## TPU Verification

In [ ]:
import os
import subprocess
import warnings
warnings.filterwarnings('ignore')

def setup_kaggle_environment():
    """Setup Kaggle environment and verify connectivity"""
    print("Setting up Kaggle environment...")

    # Check if we're in Kaggle environment
    IN_KAGGLE = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
    if IN_KAGGLE:
        print("Running in Kaggle environment")

        # Install required packages if needed
        try:
            subprocess.check_call(['pip', 'install', '--quiet', 'torch_xla'])
            print("Installed torch_xla successfully")
        except:
            print("torch_xla already installed or installation failed")

        # Reset notebook connection if needed
        try:
            from IPython.display import clear_output
            clear_output(wait=True)
            print("Cleared notebook output")
        except:
            pass

        # Set Kaggle-specific environment variables
        os.environ['XLA_USE_BF16'] = "1"
        os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

        # Verify TPU availability
        try:
            import torch_xla.core.xla_model as xm
            print("TPU backend initialized successfully")
        except ImportError:
            print("Could not initialize TPU backend")

    else:
        print("Not running in Kaggle environment")

    return IN_KAGGLE

# Initialize environment
IN_KAGGLE = setup_kaggle_environment()

# Restart kernel function
def restart_kernel():
    """Helper function to restart kernel if needed"""
    from IPython.display import Javascript
    display(Javascript('IPython.notebook.kernel.restart()'))

print("Environment setup complete. If you see connection errors, run restart_kernel()")



## GPU verification

In [ ]:
def check_gpu():
    """Check and setup GPU if available"""
    if torch.cuda.is_available():
        # Print GPU info
        print(f"CUDA is available:")
        print(f"- GPU Device: {torch.cuda.get_device_name(0)}")
        print(f"- CUDA Version: {torch.version.cuda}")
        print(f"- Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}GB")

        # Set device
        device = torch.device("cuda:0")

        # Enable optimizations
        torch.backends.cudnn.benchmark = True
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

        return device
    else:
        print("No GPU detected. Using CPU.")
        return torch.device("cpu")

## Installs

In [ ]:
# Usage in notebook
!pip install torch_xla
!pip install cloud-tpu-client
!pip install cloud-tpu-client==0.10 torch_xla==2.0 torch==2.0.0 torchvision==0.15.1 -f https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl


## Importing Liberaries

In [ ]:
import os
os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

try:
    import torch_xla.debug.metrics as met
except ImportError:
    print("Note: TPU metrics module not available")
# imoisoningdetectionmetricort requests
# requests.get('http://metadata.google.internal/computeMetadata/v1/instance/name', headers={'Metadata-Flavor': 'Google'})

In [ ]:
# Standard imports

import gc
import time
import random
import logging
import traceback
from typing import Dict, List, Tuple
from collections import deque, defaultdict

# Data processing
import numpy as np
import pandas as pd
import scipy.stats
from sklearn.model_selection import train_test_split

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

# Machine Learning
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import Sequence
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from typing import Dict, Any, List, Union

# Utilities
import psutil
from tqdm import tqdm
import contextlib
from torch.nn.functional import sigmoid

import torch.cuda
import torch.backends.cudnn

import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import (confusion_matrix, roc_auc_score, precision_score, 
                           recall_score, f1_score, average_precision_score,
                           brier_score_loss, matthews_corrcoef, cohen_kappa_score)


In [ ]:
        import torch_xla
        import torch_xla.core.xla_model as xm
        import torch_xla.distributed.parallel_loader as pl
        import torch_xla.distributed.xla_multiprocessing as xmp

## Add TPU call

In [ ]:
# TPU specific imports with error handling
def setup_tpu_imports():
    try:
        import torch_xla
        import torch_xla.core.xla_model as xm
        import torch_xla.distributed.parallel_loader as pl
        import torch_xla.distributed.xla_multiprocessing as xmp
        
        # Verify TPU is available
        devices = xm.get_xla_supported_devices()
        if devices:
            print(f"TPU devices available: {len(devices)}")
            return True, (torch_xla, xm, pl, xmp)
        else:
            print("No TPU devices found, falling back to CPU/GPU")
            return False, None
    except ImportError as e:
        print(f"TPU imports failed: {str(e)}")
        print("Falling back to CPU/GPU")
        return False, None
    except Exception as e:
        print(f"Unexpected error during TPU setup: {str(e)}")
        return False, None

def initialize_device():
    """Initialize compute device with proper error handling"""
    # Try TPU first
    tpu_available, tpu_modules = setup_tpu_imports()
    if tpu_available:
        try:
            xm = tpu_modules[1]
            device = xm.xla_device()
            print(f"Using TPU device: {device}")
            return device, tpu_modules
        except Exception as e:
            print(f"TPU initialization failed: {str(e)}")
    
    # Fall back to GPU if available
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        return device, None
    
    # Fall back to CPU
    print("Using CPU")
    return torch.device("cpu"), None





## Device Manager


In [ ]:
class DeviceManager:
    """Centralized device management for consistent handling"""
    @staticmethod
    def initialize_device():
        try:
            # Try TPU first
            import torch_xla.core.xla_model as xm
            device = xm.xla_device()
            use_tpu = True
            print("Using TPU device")
        except:
            # Fallback to GPU/CPU
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            use_tpu = False
            print(f"Using device: {device}")
        return device, use_tpu

    @staticmethod
    def to_tensor(data, device):
        """Safely convert data to tensor on specified device"""
        try:
            if isinstance(data, np.ndarray):
                return torch.from_numpy(data).to(device)
            elif isinstance(data, torch.Tensor):
                return data.to(device)
            elif isinstance(data, (list, tuple)):
                return torch.tensor(data).to(device)
            else:
                raise TypeError(f"Unsupported data type: {type(data)}")
        except Exception as e:
            print(f"Error converting to tensor: {str(e)}")
            return None

    @staticmethod
    def sync_device(device):
        """Synchronize device operations"""
        if str(device).startswith('xla'):
            import torch_xla.core.xla_model as xm
            xm.mark_step()

## Label Handling

In [ ]:
# At start of notebook
!nvidia-smi
print(f"CUDA Available: {torch.cuda.is_available()}")

In [ ]:
class LabelHandler:
    """Handle both binary and multi-class labels for poisoning detection"""
    def __init__(self):
        self.label_mapping = {}  # Store original label meanings
        self.attack_types = {}   # Store attack type categories
        self.binary_mapping = {} # Map between binary and multi-class

    def process_labels(self, labels: np.ndarray, label_names: List[str] = None) -> Tuple[np.ndarray, np.ndarray]:
        """Process labels to maintain both binary and multi-class information

        Args:
            labels: Original multi-class labels
            label_names: Optional list of label names/descriptions

        Returns:
            Tuple of (binary_labels, multi_class_labels)
        """
        unique_labels = np.unique(labels)

        # Store mapping if not already created
        if not self.label_mapping:
            self.label_mapping = {
                idx: name if label_names is not None and idx < len(label_names) else f"Class_{idx}"
                for idx in unique_labels
            }

            # Create binary mapping (0 for normal, 1 for any attack)
            self.binary_mapping = {
                idx: 0 if idx == 0 else 1  # Assuming 0 is normal traffic
                for idx in unique_labels
            }

            # Store attack types separately
            self.attack_types = {
                idx: label
                for idx, label in self.label_mapping.items()
                if idx != 0  # Exclude normal traffic
            }

        # Create both label versions
        binary_labels = np.array([self.binary_mapping[l] for l in labels])
        multi_labels = labels.copy()

        return binary_labels, multi_labels

    def get_attack_info(self, attack_id: int) -> Dict[str, Union[str, bool, int]]:
        """Get information about a specific attack type"""
        if attack_id not in self.label_mapping:
            return {
                'attack_name': f'Unknown_Attack_{attack_id}',
                'is_attack': True,
                'attack_id': attack_id,
                'binary_class': self.binary_mapping.get(attack_id, 1)
            }

        return {
            'attack_name': self.label_mapping[attack_id],
            'is_attack': self.binary_mapping.get(attack_id, 1) == 1,
            'attack_id': attack_id,
            'binary_class': self.binary_mapping.get(attack_id, 1)
        }

    def get_attack_stats(self, multi_labels: np.ndarray) -> Dict[str, Dict[str, Union[int, float, bool]]]:
        """Get statistics about attack distribution"""
        unique, counts = np.unique(multi_labels, return_counts=True)
        total = len(multi_labels)

        stats = {}
        for u, c in zip(unique, counts):
            attack_info = self.get_attack_info(u)
            stats[attack_info['attack_name']] = {
                'count': int(c),
                'percentage': float(c/total),
                'is_attack': attack_info['is_attack']
            }

        return stats

    def print_distribution(self, labels: np.ndarray):
        """Print distribution of attacks in dataset"""
        stats = self.get_attack_stats(labels)

        print("\nAttack Distribution:")
        print("-" * 50)
        print(f"{'Attack Type':<30} {'Count':>8} {'Percentage':>12}")
        print("-" * 50)

        for attack_name, info in stats.items():
            print(f"{attack_name:<30} {info['count']:>8} {info['percentage']:>11.2f}%")

    def __str__(self) -> str:
        return f"LabelHandler with {len(self.attack_types)} attack types"

    def __repr__(self) -> str:
        return f"LabelHandler(n_attacks={len(self.attack_types)}, n_labels={len(self.label_mapping)})"



## Attack type mapping

In [ ]:
# Add after LabelHandler class (around line 300)
class AttackTypeMapper:
    """Maps complete attack types for each dataset"""
    @staticmethod 
    def get_mappings():
        return {
            'cic': {
                0: 'Normal/Benign',
                1: 'DDoS',
                2: 'DoS', 
                3: 'Reconnaissance',
                4: 'Backdoor',
                5: 'SQL_Injection',
                6: 'Password_Attack',
                7: 'XSS',
                8: 'Man_in_the_Middle',
                9: 'Scanning'
            },
            'ton': {
                0: 'Normal/Benign',
                1: 'Scanning',
                2: 'DoS',
                3: 'DDoS', 
                4: 'Ransomware',
                5: 'Backdoor',
                6: 'Data_Theft',
                7: 'Keylogging',
                8: 'OS_Fingerprint',
                9: 'Service_Scan',
                10: 'Data_Exfiltration',
                11: 'SQL_Injection',
                12: 'MITM',
                13: 'Spam',
                14: 'XSS',
                15: 'Cryptojacking',
                16: 'Command_Injection',
                17: 'Rootkit',
                18: 'Trojan',
                19: 'Worm',
                20: 'Botnet',
                21: 'Malware',
                22: 'Vulnerability_Scan',
                23: 'Password_Attack',
                24: 'Privilege_Escalation',
                25: 'Protocol_Manipulation',
                26: 'Remote_Shell',
                27: 'SSL_Attack',
                28: 'Tunneling',
                29: 'Web_Attack',
                30: 'Zero_Day',
                31: 'APT',
                32: 'Code_Execution',
                33: 'Brute_Force'
            },
            'cse': {
                0: 'Normal/Benign',
                1: 'Bot',
                2: 'Brute_Force',
                3: 'DoS_Hulk',
                4: 'DoS_GoldenEye', 
                5: 'DoS_Slowloris',
                6: 'DoS_Slowhttptest',
                7: 'FTP_Patator',
                8: 'Heartbleed',
                9: 'Infiltration',
                10: 'SQL_Injection'
            }
        }
    
    

## System setup

In [ ]:
# System setup function
def setup_system():
    """Setup system checks and cleanup before training"""
    print("\n=== System Setup ===")
    
    # Verify TPU first
    if TPUHandler.is_tpu_available():
        print("TPU support verified")
    else:
        print("TPU not available, using alternative device")

    # Clear memory
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f}GB")
        print(f"CUDA Capability: {torch.cuda.get_device_capability()}")
    else:
        print("Using CPU for training")

    # Check available memory
    process = psutil.Process()
    print(f"Initial memory usage: {process.memory_info().rss/1024/1024:.2f}MB")

    # Initialize logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler('training.log'),
            logging.StreamHandler()
        ]
    )

    # Verify system requirements
    memory_gb = psutil.virtual_memory().total / (1024**3)
    if memory_gb < 8:  # Minimum 8GB required
        raise RuntimeError(f"Insufficient memory: {memory_gb:.1f}GB < 8GB required")

    # Set random seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)

    print("System setup completed successfully")
    return True

# Memory monitoring class
class MemoryMonitor:
    """Monitor system memory usage during training"""
    def __init__(self, log_interval=60):  # Log every 60 seconds
        self.log_interval = log_interval
        self.last_log_time = time.time()

        # Setup logging
        logging.basicConfig(
            filename='memory_usage.log',
            level=logging.INFO,
            format='%(asctime)s - %(message)s'
        )

    def check_memory(self):
        current_time = time.time()
        if current_time - self.last_log_time >= self.log_interval:
            process = psutil.Process()
            memory_info = process.memory_info()
            system_memory = psutil.virtual_memory()

            logging.info(
                f"Memory Usage - RSS: {memory_info.rss/1024/1024:.2f}MB, "
                f"VMS: {memory_info.vms/1024/1024:.2f}MB, "
                f"System Memory Used: {system_memory.percent}%"
            )

            self.last_log_time = current_time
            return system_memory.percent > 90  # Warning threshold
        return False

class GPUMemoryManager:
    @staticmethod
    def print_memory_stats():
        """Print memory usage for either GPU or CPU"""
        if torch.cuda.is_available():
            print("\nGPU Memory Usage:")
            print(f"Allocated: {torch.cuda.memory_allocated()/1e9:.2f}GB")
            print(f"Cached: {torch.cuda.memory_reserved()/1e9:.2f}GB")
        else:
            process = psutil.Process()
            print("\nCPU Memory Usage:")
            print(f"RSS: {process.memory_info().rss/1e9:.2f}GB")
            print(f"VMS: {process.memory_info().vms/1e9:.2f}GB")

    @staticmethod
    def clear_memory():
        """Clear memory cache"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    @staticmethod
    def get_memory_usage():
        """Get current memory usage as percentage"""
        if torch.cuda.is_available():
            return torch.cuda.memory_allocated() / torch.cuda.get_device_properties(0).total_memory
        else:
            process = psutil.Process()
            return process.memory_percent()



## TPU Handler

In [ ]:
class TPUHandler:
    """Handles TPU initialization and management"""
    @staticmethod
    def setup_tpu():
        """Initialize TPU with proper error handling"""
        try:
            import torch_xla
            import torch_xla.core.xla_model as xm
            device = xm.xla_device()
            print("TPU device initialized successfully")
            return device, True
        except (ImportError, AttributeError) as e:
            print(f"TPU initialization failed: {str(e)}")
            # Fallback to GPU/CPU
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(f"Falling back to: {device}")
            return device, False
        except Exception as e:
            print(f"Unexpected error during TPU setup: {str(e)}")
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            print(f"Falling back to: {device}")
            return device, False

    @staticmethod
    def is_tpu_available():
        """Check if TPU is available"""
        try:
            import torch_xla.core.xla_model as xm
            return True
        except:
            return False

    @staticmethod
    def sync_device(device):
        """Synchronize TPU operations if needed"""
        if str(device).startswith('xla'):
            try:
                import torch_xla.core.xla_model as xm
                xm.mark_step()
            except Exception as e:
                print(f"TPU sync failed: {str(e)}")

## Model configuration

In [ ]:
class ModelConfig:
    def __init__(self):
        """Initialize model configuration with proper device handling"""
        # Initialize device first
        self.device, self.use_tpu = DeviceManager.initialize_device()

        # Set device-specific configurations
        if self.use_tpu:
            self.batch_size = 512 if self.use_tpu else 256
            self.gradient_accumulation_steps = 2
            self.num_workers = 8
            self.use_amp = False
        elif torch.cuda.is_available():
            self.batch_size = 512
            self.num_workers = 4
            self.use_amp = True
            print(f"Using GPU: {torch.cuda.get_device_name(0)}")
        else:
            self.batch_size = 128
            self.num_workers = min(2, os.cpu_count())
            self.use_amp = False
            print("Using CPU")

        # Model Architecture
        self.feature_dim = 128
        self.hidden_dim = 256
        self.num_actions = 2
        self.num_heads = 4
        self.dropout_rate = 0.2

        # Training Parameters
        self.learning_rate = 1e-4
        self.sequence_length = 10
        self.num_epochs = 100
        self.gamma = 0.99

        # DQN Specific
        self.epsilon_start = 1.0
        self.epsilon_end = 0.01
        self.epsilon_decay = 0.995
        
        # Integration parameters
        self.dqn_weight = 0.6
        self.ac_weight = 0.4
        
        # Memory and Buffer
        self.replay_buffer_size = 10000
        self.min_replay_size = 1000
        
        # Optimization
        self.gradient_clip = 1.0
        self.warmup_steps = 1000
        self.target_update_freq = 10
        
        # Early Stopping
        self.patience = 5
        self.min_delta = 0.001
        
        # Directories
        self.checkpoint_dir = 'checkpoints'
        self.log_dir = 'logs'
        
        # Setup device-specific optimizations
        self._setup_device_specific()
        
        # Print initial configuration
        self.print_config()

    def _setup_device_specific(self):
        """Setup device-specific optimizations"""
        if self.use_tpu:
            # TPU-specific optimizations
            pass
        elif self.device.type == "cuda":
            # GPU optimizations
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            torch.backends.cudnn.benchmark = True
            self.pin_memory = True
        else:
            # CPU optimizations
            self.pin_memory = False
            if hasattr(torch, 'set_num_threads'):
                torch.set_num_threads(self.num_workers)

    def print_config(self):
        """Print the current configuration"""
        print("\nModel Configuration:")
        print(f"- Device: {self.device}")
        print(f"- TPU Available: {self.use_tpu}")
        print(f"- Feature Dimension: {self.feature_dim}")
        print(f"- Hidden Dimension: {self.hidden_dim}")
        print(f"- Batch Size: {self.batch_size}")
        print(f"- Number of Workers: {self.num_workers}")
        print(f"- Learning Rate: {self.learning_rate}")
        print(f"- Number of Epochs: {self.num_epochs}")
        print(f"- DQN Weight: {self.dqn_weight}")
        print(f"- Actor-Critic Weight: {self.ac_weight}")
        print(f"- AMP Enabled: {self.use_amp}")

        if self.device.type == "cuda":
            print(f"- CUDA Capability: {torch.cuda.get_device_capability()}")
            print(f"- GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f}GB")

    def get_device_info(self) -> Dict[str, Any]:
        """Get detailed device information"""
        info = {
            'device_type': self.device.type,
            'use_tpu': self.use_tpu,
            'use_amp': self.use_amp,
            'batch_size': self.batch_size,
            'num_workers': self.num_workers
        }

        if self.device.type == "cuda":
            info.update({
                'cuda_capability': torch.cuda.get_device_capability(),
                'gpu_name': torch.cuda.get_device_name(0),
                'gpu_memory': torch.cuda.get_device_properties(0).total_memory
            })

        return info
    

## Safe Serialization

In [ ]:
class ModelSerializer:
    """Safe model serialization and deserialization"""
    @staticmethod
    def save_model(model, path, additional_info=None):
        try:
            save_dict = {
                'model_state_dict': model.state_dict(),
                'config': ModelSerializer._prepare_config(additional_info)
            }
            torch.save(save_dict, path)
            return True
        except Exception as e:
            print(f"Error saving model: {str(e)}")
            return False

    @staticmethod
    def _prepare_config(config):
        """Prepare configuration for serialization"""
        if config is None:
            return {}
            
        def convert_to_serializable(obj):
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, dict):
                return {k: convert_to_serializable(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_to_serializable(i) for i in obj]
            elif isinstance(obj, (int, float, str, bool)):
                return obj
            else:
                return str(obj)
                
        return convert_to_serializable(config)

## Data Loading and Preprocessing

In [ ]:
## DATASET SPECIFIC PROCESSOR

class DatasetSpecificProcessor:
    """Processes features specific to each dataset type"""
    def __init__(self, dataset_type: str):
        self.dataset_type = dataset_type.lower()
        self.protocol_features = {}
        self.temporal_windows = {}
        self.flow_statistics = {}

    def process_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Process features based on dataset type"""
        # Ensure data is properly formatted as DataFrame with string column names
        if not isinstance(data, pd.DataFrame):
            data = pd.DataFrame(data)

        # Convert numeric column indices to string names if needed
        if all(isinstance(col, int) for col in data.columns):
            data.columns = [f'feature_{i}' for i in range(len(data.columns))]

        if self.dataset_type == 'cic':
            return self._process_cic_features(data)
        elif self.dataset_type == 'ton':
            return self._process_ton_features(data)
        elif self.dataset_type == 'cse':
            return self._process_cse_features(data)
        else:
            raise ValueError(f"Unknown dataset type: {self.dataset_type}")

    def _process_cic_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Process CIC-IoT specific features"""
        processed_data = data.copy()

        # Find protocol-related columns (case insensitive)
        protocol_columns = [
            col for col in processed_data.columns
            if isinstance(col, str) and 'protocol' in col.lower()
        ]

        # Process protocol features if any exist
        if protocol_columns:
            # One-hot encoding for protocol columns
            processed_data = pd.get_dummies(
                processed_data,
                columns=protocol_columns,
                prefix=['protocol']
            )

        # Calculate packet and connection rates for numeric columns
        numeric_cols = processed_data.select_dtypes(include=[np.number]).columns
        if 'packet_count' in numeric_cols and 'duration' in numeric_cols:
            processed_data['packet_rate'] = (
                processed_data['packet_count'] /
                processed_data['duration'].clip(lower=1e-6)
            )

        if 'connection_count' in numeric_cols and 'duration' in numeric_cols:
            processed_data['connection_rate'] = (
                processed_data['connection_count'] /
                processed_data['duration'].clip(lower=1e-6)
            )

        return processed_data

    def _process_ton_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Process TON-IoT specific features"""
        processed_data = data.copy()
        window_size = 10

        # Process numeric features
        numeric_cols = processed_data.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            # Calculate temporal mean and variance safely
            try:
                processed_data[f'{col}_temporal_mean'] = processed_data[col].rolling(
                    window=window_size, min_periods=1
                ).mean()
                processed_data[f'{col}_temporal_var'] = processed_data[col].rolling(
                    window=window_size, min_periods=1
                ).var()
            except Exception as e:
                print(f"Warning: Could not process column {col}: {str(e)}")

        # Find and process service-related columns
        service_columns = [
            col for col in processed_data.columns
            if isinstance(col, str) and 'service' in col.lower()
        ]

        if service_columns:
            for service in service_columns:
                try:
                    # Convert to numeric if needed
                    if not pd.api.types.is_numeric_dtype(processed_data[service]):
                        processed_data[service] = pd.to_numeric(
                            processed_data[service], errors='coerce'
                        )
                    # Calculate service rate
                    processed_data[f'{service}_rate'] = processed_data[service].rolling(
                        window=window_size, min_periods=1
                    ).mean()
                except Exception as e:
                    print(f"Warning: Could not process service column {service}: {str(e)}")

        return processed_data

    def _process_cse_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """Process CSE-CIC specific features"""
        processed_data = data.copy()
        window_size = 10

        # Process numeric features only
        numeric_cols = processed_data.select_dtypes(include=[np.number]).columns

        # Process flow-based features
        if 'bytes_transferred' in numeric_cols and 'duration' in numeric_cols:
            processed_data['flow_byte_rate'] = (
                processed_data['bytes_transferred'] /
                processed_data['duration'].clip(lower=1e-6)
            )

        # Process packet statistics
        if 'packet_size' in numeric_cols:
            processed_data['packet_size_mean'] = processed_data['packet_size'].rolling(
                window=window_size, min_periods=1
            ).mean()
            processed_data['packet_size_std'] = processed_data['packet_size'].rolling(
                window=window_size, min_periods=1
            ).std()

        # Process flow statistics
        if 'flow_duration' in numeric_cols:
            processed_data['flow_rate'] = 1.0 / processed_data['flow_duration'].clip(lower=1e-6)
            processed_data['flow_rate_mean'] = processed_data['flow_rate'].rolling(
                window=window_size, min_periods=1
            ).mean()

        return processed_data


class PoisoningFeatureExtractor:
    """Optimized poisoning feature extractor with full feature set"""
    def __init__(self, window_size: int = 100, feature_dim: int = None):
        self.window_size = window_size
        self.feature_history = deque(maxlen=window_size)
        self.distribution_history = deque(maxlen=window_size)
        self.feature_dim = feature_dim
        self.eps = 1e-8
        
    def extract_poisoning_features(self, data: pd.DataFrame) -> Dict[str, np.ndarray]:
        """Extract all poisoning-specific features with optimized computation"""
        try:
            # Ensure data is properly formatted
            if not isinstance(data, pd.DataFrame):
                data = pd.DataFrame(data)
                
            # Initialize feature dictionary
            features = {}
            
            # Extract all features with proper error handling
            features.update(self._detect_distribution_drift(data))
            features.update(self._check_temporal_consistency(data))
            features.update(self._analyze_protocol_behavior(data))
            features.update(self._detect_traffic_anomalies(data))
            
            return features
            
        except Exception as e:
            print(f"Warning: Error in feature extraction: {str(e)}")
            return {
                'distribution_drift': np.array([0.0]),
                'temporal_consistency': np.array([0.0]),
                'protocol_frequency': np.array([0.0]),
                'traffic_anomaly': np.array([0.0])
            }

    def _detect_distribution_drift(self, data: pd.DataFrame) -> Dict[str, float]:
        """Optimized distribution drift detection"""
        try:
            numeric_data = data.select_dtypes(include=[np.number])
            
            if len(self.feature_history) == 0:
                self.feature_history.append(numeric_data)
                return {'distribution_drift': np.array([0.0])}
                
            # Efficient mean computation
            current_mean = numeric_data.mean()
            previous_mean = np.nanmean([x.mean() for x in self.feature_history], axis=0)
            
            # Optimized covariance computation
            current_values = numeric_data.values
            previous_values = np.vstack([x.values for x in self.feature_history])
            
            # Add stability term to covariance matrices
            current_cov = np.cov(current_values.T) + self.eps * np.eye(current_values.shape[1])
            previous_cov = np.cov(previous_values.T) + self.eps * np.eye(previous_values.shape[1])
            
            # Calculate drift score
            mean_diff = np.linalg.norm(current_mean - previous_mean)
            cov_diff = np.linalg.norm(current_cov - previous_cov, ord='fro')
            drift_score = (mean_diff + cov_diff) / (1.0 + self.eps)
            
            self.feature_history.append(numeric_data)
            
            return {'distribution_drift': np.array([drift_score])}
            
        except Exception as e:
            print(f"Warning: Error in drift detection: {str(e)}")
            return {'distribution_drift': np.array([0.0])}

    def _check_temporal_consistency(self, data: pd.DataFrame) -> Dict[str, float]:
        """Optimized temporal consistency check"""
        try:
            numeric_data = data.select_dtypes(include=[np.number])
            
            if len(self.feature_history) == 0:
                return {'temporal_consistency': np.array([0.0])}
                
            previous_data = self.feature_history[-1]
            common_cols = set(numeric_data.columns) & set(previous_data.columns)
            
            if not common_cols:
                return {'temporal_consistency': np.array([0.0])}
                
            # Vectorized consistency calculation
            consistency_scores = np.array([
                np.abs(numeric_data[col].values - previous_data[col].values).mean()
                for col in common_cols
            ])
            
            return {'temporal_consistency': np.array([consistency_scores.mean()])}
            
        except Exception as e:
            print(f"Warning: Error in temporal consistency: {str(e)}")
            return {'temporal_consistency': np.array([0.0])}

    def _analyze_protocol_behavior(self, data: pd.DataFrame) -> Dict[str, np.ndarray]:
        """Optimized protocol behavior analysis"""
        try:
            protocol_columns = [
                col for col in data.columns
                if isinstance(col, str) and 'protocol' in col.lower()
            ]
            
            if not protocol_columns:
                return {'protocol_frequency': np.array([0.0])}
                
            # Vectorized protocol frequency calculation
            protocol_data = data[protocol_columns]
            protocol_freqs = protocol_data.mean().values
            
            # Store in distribution history for trend analysis
            self.distribution_history.append(protocol_freqs)
            
            return {'protocol_frequency': protocol_freqs}
            
        except Exception as e:
            print(f"Warning: Error in protocol analysis: {str(e)}")
            return {'protocol_frequency': np.array([0.0])}

    def _detect_traffic_anomalies(self, data: pd.DataFrame) -> Dict[str, np.ndarray]:
        """Optimized traffic anomaly detection"""
        try:
            numeric_data = data.select_dtypes(include=[np.number])
            
            if 'packet_count' not in numeric_data.columns:
                return {'traffic_anomaly': np.array([0.0])}
                
            packet_counts = numeric_data['packet_count'].values
            expected_count = np.mean(packet_counts)
            
            # Vectorized anomaly score calculation
            anomaly_scores = np.abs(
                (packet_counts - expected_count) /
                np.maximum(expected_count, self.eps)
            )
            
            return {'traffic_anomaly': anomaly_scores}
            
        except Exception as e:
            print(f"Warning: Error in anomaly detection: {str(e)}")
            return {'traffic_anomaly': np.array([0.0])}

    def get_feature_stats(self) -> Dict[str, Any]:
        """Get statistics about extracted features"""
        return {
            'window_size': len(self.feature_history),
            'distribution_window': len(self.distribution_history),
            'feature_dimensions': self.feature_dim,
            'total_samples_processed': sum(len(x) for x in self.feature_history)
        }

    def reset(self):
        """Reset extractor state"""
        self.feature_history.clear()
        self.distribution_history.clear()

    
class PoisoningGenerator:
    """Generates synthetic poisoning samples"""
    def __init__(self, epsilon: float = 0.1, flip_rate: float = 0.1):
        self.epsilon = epsilon
        self.flip_rate = flip_rate

    def generate_gradient_based_poisoning(self, data: torch.Tensor, loss_fn: callable) -> torch.Tensor:
        """Generate gradient-based poisoning samples"""
        data.requires_grad = True
        loss = loss_fn(data)
        gradient = torch.autograd.grad(loss, data)[0]
        poisoned_data = data + self.epsilon * torch.sign(gradient)
        return poisoned_data.detach()

    def generate_label_flipping_attacks(self, labels: np.ndarray) -> np.ndarray:
        """Generate label flipping attacks"""
        flipped_labels = labels.copy()
        flip_mask = np.random.random(len(labels)) < self.flip_rate
        flipped_labels[flip_mask] = 1 - labels[flip_mask]
        return flipped_labels

    def generate_backdoor_triggers(self, data: np.ndarray, trigger_pattern: np.ndarray) -> np.ndarray:
        """Generate backdoor triggers"""
        poisoned_data = data.copy()
        backdoor_mask = np.random.random(len(data)) < self.flip_rate
        poisoned_data[backdoor_mask] += trigger_pattern
        return poisoned_data

    def generate_clean_label_poisoning(self, data: np.ndarray, boundary_shift: np.ndarray) -> np.ndarray:
        """Generate clean label poisoning"""
        poisoned_data = data.copy()
        poison_mask = np.random.random(len(data)) < self.flip_rate
        poisoned_data[poison_mask] += boundary_shift
        return poisoned_data


## DATASET LOADER



class EnhancedDatasetLoader:
    """Enhanced dataset loader with dataset-specific processing and poisoning detection"""
    def __init__(self, dataset_type: str, config: ModelConfig = None):
        self.dataset_type = dataset_type.lower()
        self.config = config or ModelConfig()

        # Label column mappings
        self.label_columns = {
            'cic': 'Label',
            'ton': 'label',
            'cse': ' Label'  # Note the space before Label for CSE dataset
        }

        # Initialize processors
        self.feature_scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.label_handler = LabelHandler()
        self.dataset_processor = DatasetSpecificProcessor(self.dataset_type)
        self.poisoning_extractor = PoisoningFeatureExtractor()
        self.poisoning_generator = PoisoningGenerator()
        self.data_stabilizer = DataStabilizer()

        # Add clip values
        self.max_value = 1e10
        self.min_value = -1e10

        # Validation thresholds
        self.validation_thresholds = {
            'mean_threshold': 0.1,
            'std_threshold': 0.1,
            'correlation_threshold': 0.95,
            'min_class_ratio': 0.01
        }

        # Statistics tracking
        self.stats = defaultdict(list)
        self.feature_columns = None
        self.removed_columns = set()
        self._cached_label_column = None 

        print(f"\nInitialized Enhanced Dataset Loader for {self.dataset_type.upper()}")

    def _get_label_column(self, chunk: pd.DataFrame) -> str:
        """Get appropriate label column with caching"""
        if self._cached_label_column is None:
            # First try the predefined mapping
            default_label = self.label_columns.get(self.dataset_type)
            if default_label in chunk.columns:
                self._cached_label_column = default_label
                return default_label

            # Try common variations
            common_labels = ['Label', 'label', ' Label', 'type', 'class']
            for label in common_labels:
                if label in chunk.columns:
                    self._cached_label_column = label
                    return label

            # If still not found, look for any column containing 'label'
            label_cols = [col for col in chunk.columns if 'label' in col.lower()]
            if label_cols:
                self._cached_label_column = label_cols[0]
                return label_cols[0]

            raise ValueError(f"No label column found for {self.dataset_type} dataset")

        return self._cached_label_column        
    
        
    def _get_label_column(self, chunk: pd.DataFrame) -> str:
        """Get appropriate label column based on dataset type and available columns"""
        # First try the predefined mapping
        default_label = self.label_columns.get(self.dataset_type)
        if default_label in chunk.columns:
            return default_label

        # Try common variations
        common_labels = ['Label', 'label', ' Label', 'type', 'class']
        for label in common_labels:
            if label in chunk.columns:
                return label

        # If still not found, look for any column containing 'label' (case insensitive)
        label_cols = [col for col in chunk.columns if 'label' in col.lower()]
        if label_cols:
            return label_cols[0]

        raise ValueError(f"No label column found for {self.dataset_type} dataset")

    def _process_chunk(self, chunk: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """Process data chunk with appropriate label handling"""
        try:
            # Get label column
            label_col = self._get_label_column(chunk)
            print(f"Found label column: {label_col}")

            if label_col not in chunk.columns:
                raise ValueError(f"Label column '{label_col}' not found")

            # Convert numeric columns
            feature_cols = [col for col in chunk.columns if col != label_col]
            numeric_chunk = pd.DataFrame()

            for col in feature_cols:
                try:
                    numeric_chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
                except Exception as e:
                    print(f"Warning: Could not convert column {col}: {str(e)}")
                    numeric_chunk[col] = 0

            # Handle missing values
            numeric_chunk = numeric_chunk.fillna(0)

            # Extract features and preprocess
            features = self._preprocess_features(numeric_chunk.values)
            labels = chunk[label_col].values

            # Verify data
            if features is None or len(features) == 0:
                raise ValueError("No valid features extracted from chunk")

            return features, labels

        except Exception as e:
            print(f"Error processing chunk: {str(e)}")
            return None, None


    def _preprocess_features(self, features: np.ndarray) -> np.ndarray:
        """Preprocess features to handle infinities and large values"""
        try:
            if features is None or len(features) == 0:
                return None

            # Replace inf values
            features = np.nan_to_num(features, nan=0.0, posinf=self.max_value, neginf=self.min_value)

            # Clip extreme values
            features = np.clip(features, self.min_value, self.max_value)

            # Check for invalid values
            if not np.all(np.isfinite(features)):
                print("Warning: Invalid values found after preprocessing")
                features = np.nan_to_num(features, nan=0.0)

            return features

        except Exception as e:
            print(f"Error preprocessing features: {str(e)}")
            return None

    def process_large_dataset(self, data, batch_size=10000):
        chunks = np.array_split(data, len(data) // batch_size + 1)
        processed_chunks = []

        for chunk in chunks:
            # Process chunk with memory cleanup
            processed = self._process_chunk(chunk)
            processed_chunks.append(processed)
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        return np.concatenate(processed_chunks)

    
    def load_and_process_dataset(self, file_path: str):

        try:
            print(f"\nProcessing {self.dataset_type.upper()} dataset: {file_path}")

            if not os.path.exists(file_path):
                raise FileNotFoundError(f"Dataset file not found: {file_path}")

            chunks = []
            labels = []
            poisoning_features = defaultdict(list)
            
                
        # Replace direct processing with chunked processing
        if len(chunks) > 10000:  # Large dataset threshold
            X = self.process_large_dataset(np.vstack(chunks))
        else:
            X = np.vstack(chunks)


            # Read and process chunks
            for chunk in pd.read_csv(file_path, chunksize=10000):
                features, chunk_labels = self._process_chunk(chunk)

                if features is not None and len(features) > 0:
                    try:
                        processed_chunk = self.dataset_processor.process_features(pd.DataFrame(features))
                        processed_features = self._preprocess_features(processed_chunk.values)

                        if processed_features is not None and len(processed_features) > 0:
                            chunks.append(processed_features)
                            labels.extend(chunk_labels)

                            try:
                                poison_features = self.poisoning_extractor.extract_poisoning_features(processed_chunk)
                                for key, value in poison_features.items():
                                    if value is not None and len(value) > 0:
                                        poisoning_features[key].append(value)
                            except Exception as e:
                                print(f"Warning: Error extracting poisoning features: {str(e)}")

                    except Exception as e:
                        print(f"Warning: Error processing chunk features: {str(e)}")

            if not chunks:
                raise ValueError("No valid data chunks processed")

            # Combine and process data
            X = np.vstack(chunks)
            y = np.array(labels)

            # Scale features
            X_scaled = self.feature_scaler.fit_transform(X)

            # Process labels
            y_encoded = self.label_encoder.fit_transform(y)
            binary_labels, multi_labels = self.label_handler.process_labels(y_encoded)

            # Combine poisoning features
            combined_poison_features = {}
            for key, values in poisoning_features.items():
                if values:
                    try:
                        combined_poison_features[key] = np.concatenate(values)
                    except Exception as e:
                        print(f"Warning: Could not combine poisoning features for {key}: {str(e)}")
                        combined_poison_features[key] = np.zeros(len(X))

            # Generate synthetic poisoning samples
            poisoned_samples = self._generate_poisoning_samples(X_scaled, binary_labels)

            print("\nDataset Processing Complete:")
            print(f"- Total samples: {len(y)}")
            print(f"- Feature dimensions: {X_scaled.shape[1]}")
            print(f"- Poisoning features extracted: {list(combined_poison_features.keys())}")

            return X_scaled, binary_labels, multi_labels, {
                'poisoning_features': combined_poison_features,
                'poisoned_samples': poisoned_samples,
                'validation_stats': self._validate_data_quality(X_scaled, binary_labels, multi_labels)
            }

        except Exception as e:
            print(f"Error processing dataset: {str(e)}")
            traceback.print_exc()
            raise


    def _validate_data_quality(self, X: np.ndarray, binary_labels: np.ndarray, multi_labels: np.ndarray) -> Dict:
        """Validate data quality and compute statistics"""
        validation_stats = {
            'feature_stats': {
                'mean': np.mean(X, axis=0),
                'std': np.std(X, axis=0)
            },
            'class_distribution': {
                'binary': np.bincount(binary_labels),
                'multi': np.bincount(multi_labels)
            },
            'missing_values': np.isnan(X).sum(),
            'feature_correlations': np.corrcoef(X.T)
        }
        return validation_stats

    def _generate_poisoning_samples(self, X: np.ndarray, y: np.ndarray) -> Dict[str, np.ndarray]:
        """Generate synthetic poisoning samples"""
        X_tensor = torch.FloatTensor(X)

        def dummy_loss(x):
            return torch.mean(x ** 2)

        poisoned_samples = {
            'gradient_based': self.poisoning_generator.generate_gradient_based_poisoning(
                X_tensor, dummy_loss
            ).numpy(),
            'label_flipping': self.poisoning_generator.generate_label_flipping_attacks(y),
            'backdoor': self.poisoning_generator.generate_backdoor_triggers(
                X, np.random.normal(0, 0.1, X.shape[1])
            ),
            'clean_label': self.poisoning_generator.generate_clean_label_poisoning(
                X, np.random.normal(0, 0.1, X.shape[1])
            )
        }

        return poisoned_samples


class DatasetStatistics:
    """Track and analyze dataset statistics"""
    def __init__(self):
        self.stats = defaultdict(list)

    def update(self, batch_stats: Dict):
        """Update statistics with batch information"""
        for k, v in batch_stats.items():
            self.stats[k].append(v)

    def get_summary(self) -> Dict:
        """Get summary statistics"""
        summary = {}
        for k, v in self.stats.items():
            if isinstance(v[0], (int, float, np.number)):
                summary[k] = {
                    'mean': np.mean(v),
                    'std': np.std(v),
                    'min': np.min(v),
                    'max': np.max(v)
                }
        return summary

class BatchGenerator:
    """Generate training batches with augmentation"""
    def __init__(self, X: np.ndarray, y: np.ndarray, batch_size: int,
                 shuffle: bool = True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_samples = len(X)
        self.indices = np.arange(self.n_samples)

    def __len__(self):
        return int(np.ceil(self.n_samples / self.batch_size))

    def __iter__(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

        for start_idx in range(0, self.n_samples, self.batch_size):
            end_idx = min(start_idx + self.batch_size, self.n_samples)
            batch_indices = self.indices[start_idx:end_idx]

            yield (
                self.X[batch_indices],
                self.y[batch_indices]
            )



## neural network Component

In [ ]:
# Feature Extraction
class FeatureExtractor(nn.Module):
    """Enhanced feature extraction with attention"""
    def __init__(self, input_dim: int, hidden_dims: List[int], dropout_rate: float = 0.2):
        super().__init__()
        self.input_dim = input_dim
        layers = []
        prev_dim = input_dim

        for dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, dim),
                nn.LayerNorm(dim),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_dim = dim

        self.feature_layers = nn.Sequential(*layers)
        print(f"Feature extractor created with input dim: {input_dim}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        if x.dim() == 1:
            x = x.unsqueeze(0)
        elif x.dim() > 2:
            x = x.view(x.size(0), -1)

        return self.feature_layers(x)

class DataStabilizer:
    """Handle numerical stability in data processing"""
    def __init__(self, eps=1e-8):
        self.eps = eps

    def stabilize_array(self, arr):
        """Stabilize numpy array by handling zeros and infinities"""
        # Replace infinities with large finite numbers
        arr = np.nan_to_num(arr, nan=0.0, posinf=1e10, neginf=-1e10)
        return arr

    def safe_divide(self, numerator, denominator):
        """Safe division avoiding divide by zero"""
        return numerator / (denominator + self.eps)

    def normalize_features(self, features):
        """Normalize features with numerical stability"""
        mean = np.mean(features, axis=0)
        std = np.std(features, axis=0) + self.eps
        return (features - mean) / std

    def stabilize_gradients(self, tensor):
        """Stabilize gradients for tensor operations"""
        if torch.is_tensor(tensor):
            return torch.clamp(tensor, min=-1e6, max=1e6)
        return tensor



# DQN Component
class DQNStream(nn.Module):
    """DQN stream with LSTM and attention"""
    def __init__(self, feature_dim: int, hidden_dim: int, num_actions: int,
                 num_heads: int = 4):
        super().__init__()
        self.lstm = nn.LSTM(feature_dim, hidden_dim, batch_first=True)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads)

        self.value_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_actions)
        )

        self.advantage_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_actions)
        )

    def forward(self, features: torch.Tensor, hidden=None) -> Tuple[torch.Tensor, Tuple]:
        # LSTM processing
        lstm_out, hidden = self.lstm(features, hidden)

        # Self-attention
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)

        # Combine LSTM and attention
        combined = lstm_out + attn_out
        last_hidden = combined[:, -1, :]

        # Dueling DQN architecture
        values = self.value_head(last_hidden)
        advantages = self.advantage_head(last_hidden)

        # Combine value and advantage
        q_values = values + (advantages - advantages.mean(dim=1, keepdim=True))

        return q_values, hidden

# Actor-Critic Component
class ActorCriticStream(nn.Module):
    """Actor-Critic stream with shared features"""
    def __init__(self, feature_dim: int, hidden_dim: int, num_actions: int):
        super().__init__()

        # Shared layers
        self.shared_layer = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(hidden_dim)
        )

        # Actor (policy) network
        self.actor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_actions),
            nn.Softmax(dim=-1)
        )

        # Critic (value) network
        self.critic = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, features: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        shared_features = self.shared_layer(features)

        # Get policy distribution and state value
        action_probs = self.actor(shared_features)
        state_value = self.critic(shared_features)

        return action_probs, state_value

# Combined Architecture
class DualStreamDetector(nn.Module):
    """Integrated DQN and Actor-Critic architecture"""
    def __init__(self, input_dim: int, feature_dim: int, hidden_dim: int,
                 num_actions: int, num_heads: int = 4):
        super().__init__()

        # Components
        self.feature_extractor = FeatureExtractor(input_dim, [hidden_dim, feature_dim])
        self.dqn_stream = DQNStream(feature_dim, hidden_dim, num_actions, num_heads)
        self.ac_stream = ActorCriticStream(feature_dim, hidden_dim, num_actions)

        # Integration layer
        self.fusion_layer = nn.Sequential(
            nn.Linear(num_actions * 2, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_actions)
        )

    def forward(self, x: torch.Tensor, hidden=None) -> Dict[str, torch.Tensor]:
        # Extract features
        features = self.feature_extractor(x)

        # DQN stream
        q_values, new_hidden = self.dqn_stream(features.unsqueeze(1), hidden)

        # Actor-Critic stream
        action_probs, state_value = self.ac_stream(features)

        # Combine outputs
        combined = torch.cat([q_values, action_probs], dim=-1)
        final_output = self.fusion_layer(combined)

        return {
            'q_values': q_values,
            'action_probs': action_probs,
            'state_value': state_value,
            'final_output': final_output,
            'hidden': new_hidden
        }



## Poisoning Detection System (Base and Enhanced)

In [ ]:
class DynamicThresholdManager:
    """Manages dynamic thresholds for poisoning detection"""
    def __init__(self, initial_threshold=0.5, adaptation_rate=0.01):
        self.threshold = initial_threshold
        self.adaptation_rate = adaptation_rate
        self.historical_predictions = deque(maxlen=1000)
        self.confidence_history = deque(maxlen=1000)

    def update_threshold(self, current_confidence: float, prediction_correct: bool):
        self.historical_predictions.append(prediction_correct)
        self.confidence_history.append(current_confidence)

        recent_accuracy = np.mean(self.historical_predictions)
        confidence_variance = np.std(self.confidence_history)

        if recent_accuracy < 0.9:
            self.threshold += self.adaptation_rate * (1 - recent_accuracy)
        else:
            self.threshold -= self.adaptation_rate * confidence_variance

        self.threshold = np.clip(self.threshold, 0.3, 0.9)
        return self.threshold

## GRADUAL POISONING DETECTOR

def sigmoid(x):
    """Numpy implementation of sigmoid function"""
    return 1 / (1 + np.exp(-x))

class GradualPoisoningDetector:
    def __init__(self, window_size=100):  # Add this
        self.window_size = window_size
        self.feature_history = deque(maxlen=window_size)
        self.distribution_history = deque(maxlen=window_size)

    def analyze_gradual_changes(self, current_features: np.ndarray) -> Dict[str, float]:
        """Analyze gradual changes with numerical stability"""
        try:
            self.feature_history.append(current_features)

            if len(self.feature_history) < 2:
                return {
                    'gradual_poison_probability': 0.0,
                    'change_consistency': 0.0,
                    'change_trend': 0.0
                }

            # Calculate distribution with eps for numerical stability
            eps = 1e-8
            current_dist = np.histogram(current_features, bins=20)[0] + eps
            self.distribution_history.append(current_dist)

            if len(self.distribution_history) >= 2:
                # Use stable calculation methods
                distribution_changes = np.diff([dist for dist in self.distribution_history], axis=0)

                # Add small epsilon to avoid division by zero
                abs_changes = np.abs(distribution_changes) + eps
                mean_change = np.mean(abs_changes)
                std_change = np.std(abs_changes) + eps

                # Calculate gradual score with safety checks
                gradual_score = np.clip(
                    mean_change * std_change * len(self.distribution_history),
                    -100, 100
                )

                # Calculate trend safely
                time_points = np.arange(len(distribution_changes))
                if len(time_points) > 1:
                    try:
                        avg_changes = np.mean(distribution_changes, axis=1)
                        trend = np.polyfit(time_points, avg_changes, 1)[0]
                    except:
                        trend = 0.0
                else:
                    trend = 0.0

                return {
                    'gradual_poison_probability': float(sigmoid(gradual_score)),
                    'change_consistency': float(mean_change),
                    'change_trend': float(trend)
                }

            return {
                'gradual_poison_probability': 0.0,
                'change_consistency': 0.0,
                'change_trend': 0.0
            }

        except Exception as e:
            print(f"Warning: Error in gradual change analysis: {str(e)}")
            return {
                'gradual_poison_probability': 0.0,
                'change_consistency': 0.0,
                'change_trend': 0.0
            }


    def reset(self):
        """Reset detector state"""
        self.feature_history.clear()
        self.distribution_history.clear()

    def get_window_stats(self) -> Dict[str, float]:
        """Get statistics about the current detection window"""
        return {
            'window_size': len(self.feature_history),
            'max_window': self.window_size,
            'distribution_samples': len(self.distribution_history)
        }


## DATA AUGUMENTATION

class DataAugmentation:
    """Data augmentation techniques for poisoning detection"""
    def __init__(self, noise_std=0.01, swap_prob=0.1):
        self.noise_std = noise_std
        self.swap_prob = swap_prob

    def augment(self, data: torch.Tensor) -> torch.Tensor:
        augmented = data.clone()

        # Add Gaussian noise
        if random.random() < self.swap_prob:
            noise = torch.randn_like(augmented) * self.noise_std
            augmented += noise

        # Feature permutation
        if random.random() < self.swap_prob:
            idx = torch.randperm(augmented.size(1))
            augmented = augmented[:, idx]

        return augmented


# Metrics Tracking: 

class PoisoningDetectionMetrics:
    def __init__(self, label_handler=None):
        self.detection_history = []
        self.distribution_stats = []
        self.dqn_metrics = defaultdict(list)
        self.ac_metrics = defaultdict(list)
        self.label_handler = label_handler
        self.metrics_dict = {}
        self.last_update_time = time.time()
        self.update_interval = 1.0
        # Add these attributes
        self.true_labels = None
        self.predictions = None

    def update_metrics(self, features, prediction, true_label, dqn_values=None, ac_probs=None):
        # Store labels and predictions
        if self.true_labels is None:
            self.true_labels = []
            self.predictions = []
        self.true_labels.append(true_label)
        self.predictions.append(prediction)

        try:
            # Store detection results with feature statistics
            self.detection_history.append({
                'prediction': prediction,
                'true_label': true_label,
                'feature_stats': {
                    'mean': np.mean(features, axis=0),
                    'std': np.std(features, axis=0),
                    'kurtosis': scipy.stats.kurtosis(features, axis=0),
                    'skewness': scipy.stats.skew(features, axis=0)
                }
            })

            # Track DQN metrics
            if dqn_values is not None:
                self.dqn_metrics['q_values'].append(np.mean(dqn_values))
                self.dqn_metrics['q_std'].append(np.std(dqn_values))

            # Track Actor-Critic metrics
            if ac_probs is not None:
                self.ac_metrics['policy_entropy'].append(
                    -np.sum(ac_probs * np.log(ac_probs + 1e-10))
                )

            # Periodic computation of metrics
            current_time = time.time()
            if current_time - self.last_update_time > self.update_interval:
                self.compute_metrics()
                self.last_update_time = current_time

        except Exception as e:
            print(f"Error updating metrics: {str(e)}")

    def compute_metrics(self) -> Dict[str, float]:
        """Compute comprehensive metrics with error handling"""
        try:
            if not self.detection_history:
                return {}

            # Extract predictions and labels
            predictions = [d['prediction'] for d in self.detection_history]
            true_labels = [d['true_label'] for d in self.detection_history]

            # Convert predictions to indices if needed
            pred_indices = []
            for pred in predictions:
                if isinstance(pred, np.ndarray) and pred.ndim > 0:
                    pred_indices.append(np.argmax(pred))
                else:
                    pred_indices.append(pred)

            # Calculate basic metrics
            metrics = {
                'accuracy': np.mean([p == t for p, t in zip(pred_indices, true_labels)]),
                'detection_confidence': np.mean([
                    p.max() if isinstance(p, np.ndarray) and p.ndim > 0 else p
                    for p in predictions
                ]),
                'false_positive_rate': self._compute_fpr(pred_indices, true_labels)
            }

            # Add DQN metrics
            if self.dqn_metrics:
                metrics.update({
                    'avg_q_value': np.mean(self.dqn_metrics['q_values']),
                    'q_value_std': np.mean(self.dqn_metrics['q_std'])
                })

            # Add Actor-Critic metrics
            if self.ac_metrics:
                metrics.update({
                    'policy_entropy': np.mean(self.ac_metrics['policy_entropy'])
                })

            # Store computed metrics
            self.metrics_dict = metrics
            return metrics

        except Exception as e:
            print(f"Error computing metrics: {str(e)}")
            return {
                'accuracy': 0.0,
                'detection_confidence': 0.0,
                'false_positive_rate': 0.0
            }

    def _compute_fpr(self, predictions: List[float], true_labels: List[int]) -> float:
        """Compute False Positive Rate with error handling"""
        try:
            fp = sum(1 for p, t in zip(predictions, true_labels) if p == 1 and t == 0)
            tn = sum(1 for p, t in zip(predictions, true_labels) if p == 0 and t == 0)
            return fp / (fp + tn) if (fp + tn) > 0 else 0.0
        except Exception as e:
            print(f"Error computing FPR: {str(e)}")
            return 0.0

    def items(self):
        """Make metrics iterable"""
        if not self.metrics_dict:
            self.compute_metrics()
        return self.metrics_dict.items()

    def __getitem__(self, key):
        """Allow dictionary-like access"""
        if not self.metrics_dict:
            self.compute_metrics()
        return self.metrics_dict[key]

    def get_summary(self) -> Dict[str, Any]:
        """Get comprehensive summary of all metrics"""
        if not self.metrics_dict:
            self.compute_metrics()
        return {
            'detection_metrics': self.metrics_dict,
            'history_length': len(self.detection_history),
            'distribution_stats': len(self.distribution_stats),
            'dqn_metrics': dict(self.dqn_metrics),
            'ac_metrics': dict(self.ac_metrics)
        }

    
# Enhance Metrics Calculations

class EnhancedMetrics:
    """Advanced performance metrics calculation"""
    def __init__(self):
        self.metrics = defaultdict(dict)
        
    def calculate_metrics(self, y_true, y_pred, probabilities):
        """Calculate comprehensive set of performance metrics"""
        metrics = {
            'confusion_matrix': confusion_matrix(y_true, y_pred),
            'precision': precision_score(y_true, y_pred, average='weighted'),
            'recall': recall_score(y_true, y_pred, average='weighted'),
            'f1': f1_score(y_true, y_pred, average='weighted'),
            'matthews_corr': matthews_corrcoef(y_true, y_pred),
            'cohen_kappa': cohen_kappa_score(y_true, y_pred),
            
            # Detection and False Alarm Rates
            'detection_rate': np.sum((y_pred == 1) & (y_true == 1)) / np.sum(y_true == 1),
            'false_alarm_rate': np.sum((y_pred == 1) & (y_true == 0)) / np.sum(y_true == 0),
            
            # Additional error rates
            'false_positive_rate': np.sum((y_pred == 1) & (y_true == 0)) / np.sum(y_true == 0),
            'false_negative_rate': np.sum((y_pred == 0) & (y_true == 1)) / np.sum(y_true == 1),
            
            # Probability-based metrics
            'auc_roc': roc_auc_score(y_true, probabilities[:, 1]),
            'average_precision': average_precision_score(y_true, probabilities[:, 1]),
            'brier_score': brier_score_loss(y_true, probabilities[:, 1])
        }
        
        # Add threshold-based metrics
        thresholds = np.arange(0.1, 1.0, 0.1)
        for thresh in thresholds:
            y_pred_thresh = (probabilities[:, 1] >= thresh).astype(int)
            metrics[f'precision_at_{thresh:.1f}'] = precision_score(y_true, y_pred_thresh)
            metrics[f'recall_at_{thresh:.1f}'] = recall_score(y_true, y_pred_thresh)
            metrics[f'detection_rate_at_{thresh:.1f}'] = np.sum((y_pred_thresh == 1) & (y_true == 1)) / np.sum(y_true == 1)
            metrics[f'false_alarm_rate_at_{thresh:.1f}'] = np.sum((y_pred_thresh == 1) & (y_true == 0)) / np.sum(y_true == 0)
        
        return metrics

    def per_attack_metrics(self, y_true, y_pred, attack_mapping):
        """Calculate metrics for each attack type"""
        metrics = {}
        for attack_id, attack_name in attack_mapping.items():
            mask = y_true == attack_id
            if np.any(mask):
                y_true_masked = y_true[mask]
                y_pred_masked = y_pred[mask]
                
                metrics[attack_name] = {
                    'precision': precision_score(y_true_masked, y_pred_masked, average='binary'),
                    'recall': recall_score(y_true_masked, y_pred_masked, average='binary'),
                    'f1': f1_score(y_true_masked, y_pred_masked, average='binary'),
                    'detection_rate': np.sum((y_pred_masked == 1) & (y_true_masked == 1)) / np.sum(y_true_masked == 1),
                    'false_alarm_rate': np.sum((y_pred_masked == 1) & (y_true_masked == 0)) / np.sum(y_true_masked == 0),
                    'samples': int(np.sum(mask))
                }
        return metrics

    def get_overall_summary(self):
        """Get summary of all computed metrics"""
        summary = {}
        for dataset, metrics in self.metrics.items():
            summary[dataset] = {
                'avg_detection_rate': np.mean([m['detection_rate'] for m in metrics.values()]),
                'avg_false_alarm_rate': np.mean([m['false_alarm_rate'] for m in metrics.values()]),
                'weighted_f1': np.mean([m['f1'] for m in metrics.values()]),
                'total_samples': sum(m['samples'] for m in metrics.values())
            }
        return summary
    
    

## Base Detection Syatem

In [ ]:
# Base Detection System

class PoisoningDetectionSystem:
    """Base class for poisoning detection"""
    def __init__(self, input_dim: int, config: ModelConfig = None):
        if config is None:
            config = ModelConfig()

        self.config = config
        self.device = config.device

        # Initialize model
        self.model = DualStreamDetector(
            input_dim=input_dim,
            feature_dim=config.feature_dim,
            hidden_dim=config.hidden_dim,
            num_actions=config.num_actions
        ).to(self.device)

        # Optimizer and AMP scaler
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=config.learning_rate
        )
        self.scaler = torch.cuda.amp.GradScaler() if config.use_amp and self.device.type == "cuda" else None

        # Experience replay
        self.replay_buffer = deque(maxlen=config.replay_buffer_size)
        self.sequence_length = config.sequence_length

        # Metrics
        self.training_metrics = defaultdict(list)

    def preprocess_state(self, state: np.ndarray) -> torch.Tensor:
        """Preprocess state for model input"""
        try:
            if isinstance(state, np.ndarray):
                if state.ndim == 1:
                    state = state.reshape(1, -1)
                state_tensor = torch.from_numpy(state).float()
            else:
                state_tensor = state.float()
                if state_tensor.dim() == 1:
                    state_tensor = state_tensor.unsqueeze(0)

            return state_tensor.to(self.device)

        except Exception as e:
            print(f"Error in preprocess_state: {str(e)}")
            raise

    def detect(self, state: np.ndarray, evaluate: bool = False) -> Dict[str, np.ndarray]:
        """Detect poisoning attacks"""
        with torch.no_grad() if evaluate else torch.enable_grad():
            processed_state = self.preprocess_state(state)
            model_output = self.model(processed_state)

            # Get predictions
            q_values = model_output['q_values']
            action_probs = model_output['action_probs']
            detection_prob = F.softmax(model_output['final_output'], dim=-1)

            # Convert to numpy
            return {
                'is_poisoning': detection_prob.cpu().numpy(),
                'q_values': q_values.cpu().numpy(),
                'action_probs': action_probs.cpu().numpy(),
                'confidence': model_output['state_value'].cpu().numpy()
            }

class PoisoningLoss(nn.Module):
    def __init__(self, dqn_weight=0.4, policy_weight=0.3, value_weight=0.3, eps=1e-8):
        super().__init__()
        self.dqn_weight = dqn_weight
        self.policy_weight = policy_weight
        self.value_weight = value_weight
        self.eps = eps

        self.dqn_criterion = nn.SmoothL1Loss()
        self.policy_criterion = nn.CrossEntropyLoss()
        self.value_criterion = nn.MSELoss()

    def forward(self, model_output, targets):
        # Add numerical stability to outputs
        q_values = model_output['q_values'].clamp(min=-100, max=100)
        action_probs = F.softmax(model_output['action_probs'], dim=-1)
        action_probs = torch.clamp(action_probs, min=self.eps, max=1.0)

        # DQN loss with gradient scaling
        dqn_loss = self.dqn_criterion(q_values, targets['q_targets'])
        dqn_loss = torch.where(torch.isfinite(dqn_loss), dqn_loss, torch.zeros_like(dqn_loss))

        # Policy loss with stable log
        policy_loss = self.policy_criterion(
            action_probs,
            targets['actions']
        )

        # Value loss with bounded predictions
        value_pred = model_output['state_value'].view(-1).clamp(min=-100, max=100)
        value_target = targets['returns'].view(-1).clamp(min=-100, max=100)
        value_loss = self.value_criterion(value_pred, value_target)

        # Detection loss with stable probabilities
        detection_probs = F.softmax(model_output['final_output'], dim=-1)
        detection_probs = torch.clamp(detection_probs, min=self.eps, max=1.0)
        detection_loss = self.policy_criterion(
            detection_probs,
            targets['labels']
        )

        # Combine losses with stability checks
        total_loss = (
            self.dqn_weight * torch.nan_to_num(dqn_loss) +
            self.policy_weight * torch.nan_to_num(policy_loss + detection_loss) +
            self.value_weight * torch.nan_to_num(value_loss)
        )

        return {
            'total_loss': total_loss,
            'dqn_loss': dqn_loss.item(),
            'policy_loss': policy_loss.item(),
            'value_loss': value_loss.item(),
            'detection_loss': detection_loss.item()
        }



class PoisoningDataAugmentation:
    """Advanced data augmentation techniques specifically for poisoning detection"""
    def __init__(self,
                 noise_std=0.01,
                 feature_swap_prob=0.1,
                 feature_scale_range=(0.95, 1.05),
                 temporal_shift_prob=0.1,
                 max_shift=3):
        self.noise_std = noise_std
        self.feature_swap_prob = feature_swap_prob
        self.feature_scale_range = feature_scale_range
        self.temporal_shift_prob = temporal_shift_prob
        self.max_shift = max_shift

    def augment(self, data: torch.Tensor) -> torch.Tensor:
        """
        Apply various augmentation techniques to the input data

        Args:
            data: Input tensor of shape (batch_size, feature_dim) or (batch_size, sequence_length, feature_dim)

        Returns:
            Augmented tensor of the same shape
        """
        augmented = data.clone()

        # Add Gaussian noise
        if random.random() < self.feature_swap_prob:
            noise = torch.randn_like(augmented) * self.noise_std
            augmented += noise

        # Random feature scaling
        if random.random() < self.feature_swap_prob:
            scale_factors = torch.FloatTensor(augmented.shape[-1]).uniform_(*self.feature_scale_range)
            if augmented.dim() == 3:  # Sequential data
                scale_factors = scale_factors.unsqueeze(0).unsqueeze(0)
            else:  # Single timestep data
                scale_factors = scale_factors.unsqueeze(0)
            augmented *= scale_factors.to(augmented.device)

        # Feature permutation
        if random.random() < self.feature_swap_prob:
            feat_idx = torch.randperm(augmented.shape[-1])
            if augmented.dim() == 3:
                augmented = augmented[:, :, feat_idx]
            else:
                augmented = augmented[:, feat_idx]

        # Temporal shift for sequential data
        if augmented.dim() == 3 and random.random() < self.temporal_shift_prob:
            shift = random.randint(-self.max_shift, self.max_shift)
            augmented = torch.roll(augmented, shifts=shift, dims=1)

        # Ensure values stay within reasonable bounds
        augmented = torch.clamp(augmented, min=-10, max=10)

        return augmented

    def augment_batch(self, data: torch.Tensor, labels: torch.Tensor = None) -> tuple:
        """
        Augment a batch of data with optional label preservation

        Args:
            data: Input tensor
            labels: Optional label tensor

        Returns:
            Tuple of (augmented_data, labels)
        """
        augmented_data = self.augment(data)

        if labels is not None:
            return augmented_data, labels
        return augmented_data

    @staticmethod
    def mix_samples(data: torch.Tensor, labels: torch.Tensor, alpha: float = 0.2) -> tuple:
        """
        Implement mixup augmentation for robust learning

        Args:
            data: Input tensor
            labels: Label tensor
            alpha: Mixup interpolation strength

        Returns:
            Tuple of (mixed_data, mixed_labels)
        """
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1

        batch_size = data.size(0)
        index = torch.randperm(batch_size).to(data.device)

        mixed_data = lam * data + (1 - lam) * data[index]
        mixed_labels = lam * labels + (1 - lam) * labels[index]

        return mixed_data, mixed_labels



## Enhanced Detection System

In [ ]:
# ENHANCED DETECTION SYSTEM

class EnhancedPoisoningDetectionSystem(PoisoningDetectionSystem):
    def __init__(self, input_dim: int, config: ModelConfig = None, label_handler: LabelHandler = None):
        super().__init__(input_dim, config)

class EnhancedPoisoningDetectionSystem(PoisoningDetectionSystem):
    def __init__(self, input_dim: int, config: ModelConfig = None, label_handler: LabelHandler = None):
        super().__init__(input_dim, config)
        
        self.device = config.device if config else DeviceManager.initialize_device()[0]
        self.model = self.model.to(self.device)
        self.label_handler = label_handler

        # Initialize components
        self.label_handler = label_handler
        self.metrics_tracker = PoisoningDetectionMetrics(label_handler)
        self.threshold_manager = DynamicThresholdManager()
        self.gradual_detector = GradualPoisoningDetector()
        self.pattern_memory = deque(maxlen=1000)

        # Add missing components that caused errors
        self.data_stabilizer = DataStabilizer()
        self.criterion = PoisoningLoss()  # Initialize loss function
        self.data_augmentation = PoisoningDataAugmentation()  # Initialize data augmentation

        # Modify optimizer
        self.optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=1e-4,
            eps=1e-8  # Increased epsilon for optimizer stability
        )

        # Print initialization info
        print("\nEnhanced Detection System Initialized:")
        print(f"- Input dimension: {input_dim}")
        print(f"- Number of attack types: {len(label_handler.attack_types) if label_handler else 'N/A'}")
        print(f"- Binary classification: Normal vs Attack")
        print(f"- Using label handler: {label_handler is not None}")
        print(f"- Device: {self.device}")

        if self.device.type == "cuda":
            print(f"- GPU Memory Usage: {torch.cuda.memory_allocated()/1e9:.2f}GB")



    # Add this method to EnhancedPoisoningDetectionSystem
    def preprocess_batch(self, states, binary_labels, multi_labels):
        """Preprocess batch data with stability checks"""
        # Stabilize states
        states = self.data_stabilizer.stabilize_array(states)

        # Convert to tensors with proper dtype
        states_tensor = torch.FloatTensor(states).to(self.device)
        binary_labels_tensor = torch.LongTensor(binary_labels).to(self.device)

        # Gradient scaling for large values
        if states_tensor.abs().max() > 1e3:
            states_tensor = F.normalize(states_tensor, dim=1)

        return states_tensor, binary_labels_tensor

        # Loss function
        self.criterion = PoisoningLoss()

        # Data augmentation
        self.data_augmentation = PoisoningDataAugmentation()

        # Print initialization info
        print("\nEnhanced Detection System Initialized:")
        print(f"- Input dimension: {input_dim}")
        print(f"- Number of attack types: {len(label_handler.attack_types) if label_handler else 'N/A'}")
        print(f"- Binary classification: Normal vs Attack")
        print(f"- Using label handler: {label_handler is not None}")
        print(f"- Device: {self.device}")

        if self.device.type == "cuda":
            print(f"- GPU Memory Usage: {torch.cuda.memory_allocated()/1e9:.2f}GB")


    def _check_label_handler(self):
        """Verify label handler is available when needed"""
        if self.label_handler is None:
            raise ValueError("Label handler is required for multi-class attack analysis")
        return True

    def get_attack_info(self, attack_id: int) -> Dict:
        """Get information about a specific attack type"""
        self._check_label_handler()
        return self.label_handler.get_attack_info(attack_id)


    def detect(self, state: np.ndarray, evaluate: bool = False,
              labels: np.ndarray = None) -> Dict[str, np.ndarray]:
        """Enhanced detection with multiple analysis streams"""
        with torch.no_grad() if evaluate else torch.enable_grad():

            with torch.amp.autocast('cuda') if self.scaler else contextlib.nullcontext():
                # Process state
                processed_state = self.preprocess_state(state)
                if not evaluate and random.random() < 0.3:
                    processed_state = self.data_augmentation.augment(processed_state)

                # Model forward pass
                model_output = self.model(processed_state)
                combined_output = model_output['final_output']
                detection_prob = F.softmax(combined_output, dim=-1)

                # Get numpy state for gradual analysis
                numpy_state = state.cpu().numpy() if torch.is_tensor(state) else state

                try:
                    # Analyze gradual changes with error handling
                    gradual_analysis = self.gradual_detector.analyze_gradual_changes(numpy_state)
                except Exception as e:
                    print(f"Warning: Error in gradual analysis: {str(e)}")
                    gradual_analysis = {
                        'gradual_poison_probability': 0.0,
                        'change_consistency': 0.0,
                        'change_trend': 0.0
                    }

                # Update threshold
                current_threshold = self.threshold_manager.update_threshold(
                    detection_prob.detach().mean().item(),
                    detection_prob.detach().argmax().item() == 1
                )

                # Combine detections
                enhanced_prob = (
                    detection_prob.detach().cpu().numpy() * self.config.dqn_weight +
                    gradual_analysis['gradual_poison_probability'] * self.config.ac_weight
                )

                # Update pattern memory
                self.pattern_memory.append({
                    'features': state.detach().cpu().numpy() if torch.is_tensor(state) else state,
                    'basic_detection': detection_prob.detach().cpu().numpy(),
                    'gradual_score': gradual_analysis['gradual_poison_probability']
                })

                sequence_analysis = self._analyze_sequential_patterns()

                return {
                    'is_poisoning': enhanced_prob,
                    'q_values': model_output['q_values'].detach().cpu().numpy(),
                    'action_probs': model_output['action_probs'].detach().cpu().numpy(),
                    'confidence': model_output['state_value'].detach().cpu().numpy(),
                    'gradual_metrics': gradual_analysis,
                    'sequence_metrics': sequence_analysis,
                    'threshold': current_threshold,
                    'detection_metrics': self.metrics_tracker.compute_metrics()
                }

    def _analyze_sequential_patterns(self) -> Dict[str, float]:
        """Analyze temporal patterns in detection history"""
        try:
            if len(self.pattern_memory) < 2:
                return {
                    'sequence_score': 0.0,
                    'pattern_consistency': 0.0,
                    'temporal_correlation': 0.0
                }

            # Get recent patterns and ensure they're the same shape
            recent_patterns = list(self.pattern_memory)[-10:]
            features_list = []

            # Handle variable-sized features
            base_shape = None
            for pattern in recent_patterns:
                features = pattern['features']
                if isinstance(features, torch.Tensor):
                    features = features.cpu().numpy()

                # If this is first valid shape, use it as base
                if base_shape is None and features is not None:
                    base_shape = features.shape

                # Only include features matching base shape
                if base_shape is not None and features is not None and features.shape == base_shape:
                    features_list.append(features)

            # If we don't have enough valid patterns, return default values
            if len(features_list) < 2:
                return {
                    'sequence_score': 0.0,
                    'pattern_consistency': 0.0,
                    'temporal_correlation': 0.0
                }

            # Calculate metrics only on valid patterns
            try:
                # Convert to numpy array and calculate differences
                features_array = np.stack(features_list)
                feature_evolution = np.diff(features_array, axis=0)

                # Calculate pattern metrics
                pattern_consistency = np.mean(np.abs(feature_evolution), axis=0)

                # Get detection scores
                detection_scores = [
                    float(np.mean(p['basic_detection']))
                    for p in recent_patterns[-len(features_list):]
                ]

                gradual_scores = [
                    float(p['gradual_score'])
                    for p in recent_patterns[-len(features_list):]
                ]

                # Calculate correlation if we have enough samples
                if len(detection_scores) > 1:
                    temporal_correlation = np.corrcoef(
                        detection_scores,
                        gradual_scores
                    )[0, 1]
                    if np.isnan(temporal_correlation):
                        temporal_correlation = 0.0
                else:
                    temporal_correlation = 0.0

                sequence_score = float(sigmoid(temporal_correlation * np.mean(pattern_consistency)))

                return {
                    'sequence_score': sequence_score,
                    'pattern_consistency': float(np.mean(pattern_consistency)),
                    'temporal_correlation': float(temporal_correlation)
                }

            except Exception as e:
                print(f"Warning: Error in sequence analysis calculations: {str(e)}")
                return {
                    'sequence_score': 0.0,
                    'pattern_consistency': 0.0,
                    'temporal_correlation': 0.0
                }

        except Exception as e:
            print(f"Warning: Error in sequence pattern analysis: {str(e)}")
            return {
                'sequence_score': 0.0,
                'pattern_consistency': 0.0,
                'temporal_correlation': 0.0
            }


    def train(self, batch_size: int) -> Dict[str, float]:
        """Train the model using experiences from the replay buffer"""
        try:
            if len(self.replay_buffer) < batch_size:
                return {'status': 'insufficient_samples'}

            # Sample and prepare batch
            indices = np.random.choice(len(self.replay_buffer), batch_size, replace=False)
            batch = [self.replay_buffer[i] for i in indices]

            # Unpack and move to device
            states, binary_labels, multi_labels, predictions, next_states, dones = zip(*batch)

            states = torch.FloatTensor(np.array(states)).to(self.device)
            binary_labels = torch.LongTensor(np.array(binary_labels)).to(self.device)
            next_states = torch.FloatTensor(np.array(next_states)).to(self.device)
            dones = torch.FloatTensor(np.array(dones)).to(self.device)


            # Validate tensor shapes
            if states.dim() != 2 or next_states.dim() != 2:
                raise ValueError(f"Invalid state tensor dimensions: states={states.shape}, next_states={next_states.shape}")
            if binary_labels.dim() != 1:
                raise ValueError(f"Invalid labels tensor dimension: {binary_labels.shape}")

        except Exception as e:
            print(f"Error converting to tensors: {str(e)}")
            print(f"States shape: {np.array(states).shape if isinstance(states, (list, np.ndarray)) else 'invalid'}")
            print(f"Labels shape: {np.array(binary_labels).shape if isinstance(binary_labels, (list, np.ndarray)) else 'invalid'}")
            return {'error': 'tensor_conversion', 'details': str(e)}

        # Zero gradients
        self.optimizer.zero_grad()

        # Compute outputs and loss
        try:
          with torch.amp.autocast('cuda') if self.scaler else contextlib.nullcontext():
            # Forward pass
              outputs = self.model(states)
              next_outputs = self.model(next_states)

                    # Prepare targets for DQN
              next_q_values = next_outputs['q_values'].detach()
              q_targets = outputs['q_values'].clone().detach()
              for i in range(batch_size):
                  if not dones[i]:
                     q_targets[i, binary_labels[i]] = self.config.gamma * next_q_values[i].max()

                    # Prepare targets for actor-critic
              returns = torch.zeros_like(outputs['state_value'])
              for i in range(batch_size):
                  returns[i] = outputs['state_value'][i] + \
                              (1 - dones[i]) * self.config.gamma * next_outputs['state_value'][i].detach()

                    # Compute loss
              targets = {
                  'q_targets': q_targets,
                  'actions': binary_labels,
                  'returns': returns,
                  'labels': binary_labels
              }

              loss_dict = self.criterion(outputs, targets)
              total_loss = loss_dict['total_loss']

                    # Check for invalid loss values
              if torch.isnan(total_loss) or torch.isinf(total_loss):
                raise ValueError(f"Invalid loss value: {total_loss.item()}")

        except Exception as e:
            print(f"Error in forward pass or loss computation: {str(e)}")
            print(f"Model outputs shape: {outputs['q_values'].shape if 'q_values' in outputs else 'invalid'}")
            return {'error': 'forward_pass', 'details': str(e)}

              # Backward pass with AMP if available
        try:
            if self.scaler:
              self.scaler.scale(total_loss).backward()
              self.scaler.step(self.optimizer)
              self.scaler.update()
            else:
                total_loss.backward()
                      # Clip gradients
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.gradient_clip)
                self.optimizer.step()
        except Exception as e:
                  print(f"Error in backward pass or optimization: {str(e)}")
                  return {'error': 'backward_pass', 'details': str(e)}

              # Return metrics
        try:
            metrics = {
                 'status': 'success',
                 'total_loss': total_loss.item(),
                 'dqn_loss': loss_dict['dqn_loss'],
                 'policy_loss': loss_dict['policy_loss'],
                 'value_loss': loss_dict['value_loss'],
                 'detection_loss': loss_dict['detection_loss']
            }
            return metrics

        except Exception as e:
            print(f"Error computing metrics: {str(e)}")
            return {'error': 'metrics_computation', 'details': str(e)}

        except Exception as e:
            print(f"Unexpected error during training step: {str(e)}")
            traceback.print_exc()  # Print full traceback for debugging
            return {
                  'error': 'unexpected',
                  'details': str(e),
                  'traceback': traceback.format_exc()
              }


    def _compute_td_error(self, state_values, next_state_values, rewards, dones):
        """Compute TD error for value function updates"""
        target_values = rewards + (1 - dones) * self.config.gamma * next_state_values
        td_error = target_values - state_values
        return td_error


# Multi - Dataset Generator

In [ ]:
class MultiDatasetGenerator(Sequence):
    """Generate batches from multiple datasets with iteration control"""
    def __init__(self, datasets: Dict[str, Tuple[np.ndarray, np.ndarray, np.ndarray]],
                 batch_size=256, shuffle=True, num_workers=0):
        # Initialize dataset parameters
        self.datasets = datasets
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_workers = num_workers

        # Initialize indices for each dataset
        self.indices = {k: np.arange(len(v[0])) for k, v in datasets.items()}

        # Calculate weights for dataset sampling
        total_samples = sum(len(v[0]) for v in datasets.values())
        self.dataset_weights = {k: len(v[0])/total_samples for k, v in datasets.items()}

        # Initialize iteration control
        self.current_epoch_iterations = 0
        self.max_epoch_iterations = 1000  # Maximum iterations per epoch

        print(f"Initialized MultiDatasetGenerator with {len(datasets)} datasets")
        for k, v in datasets.items():
            print(f"- {k}: {len(v[0])} samples")

    def __iter__(self):
        """Reset iteration state and shuffle if needed"""
        self.current_epoch_iterations = 0
        if self.shuffle:
            for k in self.indices:
                np.random.shuffle(self.indices[k])
        return self

    def __next__(self):
        """Get next batch with iteration limit check"""
        if self.current_epoch_iterations >= self.max_epoch_iterations:
            raise StopIteration

        self.current_epoch_iterations += 1

        # Select dataset and get batch
        chosen_dataset = np.random.choice(
            list(self.datasets.keys()),
            p=list(self.dataset_weights.values())
        )

        X, binary_labels, multi_labels = self.datasets[chosen_dataset]
        indices = self.indices[chosen_dataset]

        # Calculate batch indices
        start_idx = (self.current_epoch_iterations * self.batch_size) % len(indices)
        batch_indices = indices[start_idx:start_idx + self.batch_size]

        # Return batch data
        return (
            X[batch_indices],
            binary_labels[batch_indices],
            multi_labels[batch_indices]
        )

    def __getitem__(self, index):
        """Get specific batch by index"""
        # Ensure index is within bounds
        if index >= self.max_epoch_iterations:
            raise IndexError("Batch index out of range")

        chosen_dataset = np.random.choice(
            list(self.datasets.keys()),
            p=list(self.dataset_weights.values())
        )

        X, binary_labels, multi_labels = self.datasets[chosen_dataset]
        indices = self.indices[chosen_dataset]

        start_idx = (index * self.batch_size) % len(indices)
        batch_indices = indices[start_idx:start_idx + self.batch_size]

        return (
            X[batch_indices],
            binary_labels[batch_indices],
            multi_labels[batch_indices]
        )

    def __len__(self):
        """Returns precise number of batches per epoch"""
        return min(
            self.max_epoch_iterations,
            int(np.ceil(sum(len(v[0]) for v in self.datasets.values()) / self.batch_size))
        )

    def reset(self):
        """Reset iteration state"""
        self.current_epoch_iterations = 0
        if self.shuffle:
            for k in self.indices:
                np.random.shuffle(self.indices[k])

    def get_progress(self):
        """Get training progress information"""
        return {
            'current_iteration': self.current_epoch_iterations,
            'max_iterations': self.max_epoch_iterations,
            'progress': self.current_epoch_iterations / self.max_epoch_iterations
        }


## Training Pipeline

In [ ]:
## EARLY STOPPING

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.001, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_weights = None
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.min_improvement = 1e-4
        self.max_epochs_without_improvement = 20  # Increased from 10
        self.epochs_without_improvement = 0

        
    def __call__(self, val_score, model):
        if self.best_score is None:
            self.best_score = val_score
            self.best_weights = model.state_dict()
        elif val_score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        else:
            self.best_score = val_score
            self.best_weights = model.state_dict()
            self.counter = 0
        return False
    
    def _get_model_weights(self, model) -> dict:
        """Get a deep copy of model weights"""
        return {
            name: param.cpu().clone().detach()
            for name, param in model.state_dict().items()
        }

    def restore_weights(self, model) -> None:
        """Restore model to best weights"""
        if self.restore_best_weights and self.best_weights is not None:
            model.load_state_dict(self.best_weights)

    def reset(self) -> None:
        """Reset early stopping state"""
        self.best_score = None
        self.counter = 0
        self.early_stop = False
        self.best_weights = None

    def get_best_score(self) -> float:
        """Return the best score achieved"""
        return self.best_score if self.best_score is not None else float('-inf')

    def is_best_epoch(self, val_score: float) -> bool:
        """Check if current epoch achieved best score"""
        return self.best_score is None or val_score > self.best_score + self.min_delta



## Single DataSet Trainer

In [ ]:
## SINGLE DATASET TRAINER

class SingleDatasetTrainer:
    """Enhanced trainer with comprehensive poisoning detection capabilities"""
    def __init__(self, config: ModelConfig, dataset_type: str):
        self.config = config
        self.dataset_type = dataset_type.lower()
        
        # Initialize device with TPU support
        self.device = config.device
        print(f"\nInitializing trainer with device: {self.device}")

        # Initialize components with enhanced error handling
        print(f"\nInitializing loader for {self.dataset_type.upper()}")
        self.loader = EnhancedDatasetLoader(
            dataset_type=self.dataset_type, 
            config=self.config
        )
        self.memory_monitor = MemoryMonitor()
        self.metrics_tracker = PoisoningDetectionMetrics(
            label_handler=self.loader.label_handler
        )

        # Training setup with proper directory handling
        self.writer = SummaryWriter(f'logs/{self.dataset_type}')
        self.checkpoint_dir = os.path.join(config.checkpoint_dir, self.dataset_type)
        os.makedirs(self.checkpoint_dir, exist_ok=True)
        
        # Initialize poisoning-specific components
        self.poisoning_stats = defaultdict(list)
        self.detection_history = []

    def train_on_dataset(self, file_path: str):
        """Complete training process with enhanced poisoning detection"""
        try:
            print(f"\nStarting training process for {self.dataset_type.upper()}")

            # Verify file exists and has correct format
            if not os.path.exists(file_path):
                raise FileNotFoundError(f"Dataset file not found: {file_path}")
            
            if not file_path.endswith('.csv'):
                print("Warning: Expected CSV file format")

            # Load and process dataset with poisoning features
            X, binary_labels, multi_labels, additional_info = self.loader.load_and_process_dataset(file_path)

            # Log dataset statistics
            print("\nDataset Statistics:")
            print(f"- Total samples: {len(X)}")
            print(f"- Feature dimensions: {X.shape[1]}")
            print(f"- Poisoning features extracted: {list(additional_info['poisoning_features'].keys())}")

            print("\nSplitting data into train and validation sets...")
            try:
                # Ensure array types with validation
                X = np.array(X, dtype=np.float32)
                binary_labels = np.array(binary_labels, dtype=np.int32)
                
                # Validate data before splitting
                if np.any(np.isnan(X)) or np.any(np.isinf(X)):
                    print("Warning: Data contains NaN or Inf values, cleaning...")
                    X = np.nan_to_num(X, nan=0.0, posinf=1e10, neginf=-1e10)

                # Stratified split with poisoning preservation
                X_train, X_val, y_train, y_val = train_test_split(
                    X, binary_labels,
                    test_size=0.2,
                    random_state=42,
                    stratify=binary_labels
                )
                
                # Verify split maintains poisoning distribution
                train_poison_ratio = np.mean(y_train)
                val_poison_ratio = np.mean(y_val)
                print(f"Train set poisoning ratio: {train_poison_ratio:.3f}")
                print(f"Validation set poisoning ratio: {val_poison_ratio:.3f}")
                
                print(f"Train set size: {len(X_train)}, Validation set size: {len(X_val)}")

            except Exception as e:
                print(f"Error in data splitting: {str(e)}")
                raise

            # Initialize detection system with poisoning capabilities
            detection_system = EnhancedPoisoningDetectionSystem(
                input_dim=X.shape[1],
                config=self.config,
                label_handler=self.loader.label_handler
            )
            print("\nInitialized detection system")

            # Create data generators with poisoning awareness
            train_generator = MultiDatasetGenerator(
                {self.dataset_type: (X_train, y_train, y_train)},
                batch_size=self.config.batch_size,
                shuffle=True  # Important for poisoning detection
            )

            val_generator = MultiDatasetGenerator(
                {self.dataset_type: (X_val, y_val, y_val)},
                batch_size=self.config.batch_size,
                shuffle=False  # Keep validation set ordered
            )
            print("Created data generators")

            # Setup comprehensive training pipeline
            pipeline = ComprehensiveTrainingPipeline(
                detection_system=detection_system,
                data_generator=train_generator,
                val_generator=val_generator,
                config=self.config,
                label_handler=self.loader.label_handler
            )

            # Train model with poisoning detection
            print(f"\nStarting training on {self.dataset_type.upper()}...")
            training_success = pipeline.train()

            if not training_success:
                print("Warning: Training completed with some errors")

            # Save results with enhanced poisoning information
            self._save_results(detection_system, additional_info)

            # Compute final poisoning statistics
            final_stats = self._compute_poisoning_stats(detection_system, X_val, y_val)
            print("\nFinal Poisoning Detection Statistics:")
            for metric, value in final_stats.items():
                print(f"- {metric}: {value:.4f}")

            return detection_system, self.metrics_tracker

        except Exception as e:
            print(f"Error in training process: {str(e)}")
            traceback.print_exc()
            raise

    def _save_results(self, detection_system: EnhancedPoisoningDetectionSystem, additional_info: Dict):
        """Save results with comprehensive poisoning information"""
        try:
            results_path = os.path.join(self.checkpoint_dir, f'{self.dataset_type}_results.pt')
            
            # Ensure directory exists
            os.makedirs(os.path.dirname(results_path), exist_ok=True)

            # Convert metrics to serializable format
            metrics_dict = self.metrics_tracker.compute_metrics()
            
            # Prepare poisoning features
            poisoning_info = {
                'features': additional_info['poisoning_features'],
                'detection_stats': self.poisoning_stats,
                'validation_metrics': additional_info['validation_stats']
            }

            # Save everything with proper error handling
            save_dict = {
                'model_state': detection_system.model.state_dict(),
                'config': {k: v for k, v in self.config.__dict__.items() 
                          if not k.startswith('_')},
                'metrics': metrics_dict,
                'poisoning_info': poisoning_info
            }

            # Convert numpy arrays to lists for serialization
            save_dict = self._prepare_for_saving(save_dict)

            try:
                torch.save(save_dict, results_path)
                print(f"\nResults saved to {results_path}")
            except Exception as e:
                # Fallback to NPZ format if torch.save fails
                np_path = results_path + '.npz'
                np.savez(np_path, **save_dict)
                print(f"\nResults saved as NPZ to {np_path}")

        except Exception as e:
            print(f"Error saving results: {str(e)}")
            raise

    def _prepare_for_saving(self, data: Dict) -> Dict:
        """Prepare data for saving by converting numpy arrays to lists"""
        if isinstance(data, np.ndarray):
            return data.tolist()
        elif isinstance(data, dict):
            return {k: self._prepare_for_saving(v) for k, v in data.items()}
        elif isinstance(data, list):
            return [self._prepare_for_saving(item) for item in data]
        return data

    def _compute_poisoning_stats(self, detection_system, X_val, y_val) -> Dict[str, float]:
        """Compute comprehensive poisoning detection statistics"""
        try:
            with torch.no_grad():
                # Convert to tensors
                X_val_tensor = torch.FloatTensor(X_val).to(self.device)
                
                # Get predictions
                outputs = detection_system.detect(X_val_tensor)
                predictions = outputs['is_poisoning']
                
                # Compute metrics
                return {
                    'detection_accuracy': np.mean(predictions.argmax(axis=1) == y_val),
                    'detection_confidence': np.mean(np.max(predictions, axis=1)),
                    'false_positive_rate': np.mean((predictions.argmax(axis=1) == 1) & (y_val == 0)),
                    'false_negative_rate': np.mean((predictions.argmax(axis=1) == 0) & (y_val == 1))
                }
        except Exception as e:
            print(f"Error computing poisoning stats: {str(e)}")
            return {}

        

## Compregensive Training Pipeline

In [ ]:
## COMPREHENSIVE TRAINING PIPELINE

class ComprehensiveTrainingPipeline:
    """Advanced training pipeline with monitoring and evaluation"""
    def __init__(self, detection_system: EnhancedPoisoningDetectionSystem,
                 data_generator: MultiDatasetGenerator,
                 val_generator: MultiDatasetGenerator,
                 config: ModelConfig,
                 label_handler: LabelHandler):
        # Core components (keep existing)
        self.detection_system = detection_system
        self.data_generator = data_generator
        self.val_generator = val_generator
        self.config = config
        self.label_handler = label_handler

        # Training parameters (keep existing)
        self.batch_size = config.batch_size
        self.num_epochs = config.num_epochs
        self.checkpoint_dir = config.checkpoint_dir
        self.log_dir = config.log_dir

        # Initialize trackers (keep existing)
        self._global_step = 0
        self._epoch = 0
        self.best_metrics = {
            'accuracy': 0,
            'f1_score': 0,
            'unknown_detection_rate': 0,
            'per_attack_f1': defaultdict(float)
        }

        # Setup components (new method)
        self._setup_components()

    def preprocess_batch(self, states, binary_labels, multi_labels):
        """Preprocess batch data with stability checks"""
        try:
            # Get data stabilizer from detection system
            data_stabilizer = self.detection_system.data_stabilizer

            # Stabilize states
            states = data_stabilizer.stabilize_array(states)

            # Convert to tensors with proper dtype
            states_tensor = torch.FloatTensor(states).to(self.detection_system.device)
            binary_labels_tensor = torch.LongTensor(binary_labels).to(self.detection_system.device)

            # Gradient scaling for large values
            if states_tensor.abs().max() > 1e3:
                states_tensor = F.normalize(states_tensor, dim=1)

            return states_tensor, binary_labels_tensor

        except Exception as e:
            print(f"Error in batch preprocessing: {str(e)}")
            print(f"States shape: {states.shape if isinstance(states, np.ndarray) else 'invalid'}")
            print(f"Labels shape: {binary_labels.shape if isinstance(binary_labels, np.ndarray) else 'invalid'}")
            raise



    def _setup_components(self):
        """Initialize training components while preserving existing functionality"""
        try:
            # Create directories
            os.makedirs(self.checkpoint_dir, exist_ok=True)
            os.makedirs(self.log_dir, exist_ok=True)

            # Initialize monitoring (preserve existing)
            self.writer = SummaryWriter(self.log_dir)
            self.memory_monitor = MemoryMonitor()
            self.early_stopping = EarlyStopping(
                patience=self.config.patience,
                min_delta=self.config.min_delta
            )

            # Initialize attack-specific monitoring
            self.attack_metrics = {
                attack_id: defaultdict(list)
                for attack_id in self.label_handler.attack_types
            }

            # Performance tracking (integrates with existing metrics)
            self.train_losses = []
            self.val_losses = []
            self.performance_history = defaultdict(list)
            self.progress_monitor = ProgressMonitor(self.log_dir)

            # Loss function (preserve existing criterion)
            self.criterion = PoisoningLoss()

            print(f"\nPipeline initialized:")
            print(f"- Batch size: {self.batch_size}")
            print(f"- Epochs: {self.num_epochs}")
            print(f"- Checkpoints: {self.checkpoint_dir}")
            print(f"- Logs: {self.log_dir}")
            print(f"- Monitoring {len(self.label_handler.attack_types)} attack types")

        except Exception as e:
            print(f"Error setting up training components: {str(e)}")
            raise
            
    
    def train(self):
        """Execute training loop with safe TPU handling and enhanced data processing"""
        try:
            # Get device from detection system
            device = self.detection_system.device
            print(f"\nStarting training on device: {device}")

            # Safe TPU check and loader creation
            use_tpu = is_tpu_device(device)
            if use_tpu:
                try:
                    import torch_xla.distributed.parallel_loader as pl
                    loader = pl.ParallelLoader(self.data_generator, [device]).per_device_loader(device)
                except Exception as e:
                    print(f"Warning: TPU parallel loading failed, using default loader: {str(e)}")
                    loader = self.data_generator
            else:
                loader = self.data_generator

            for epoch in range(self.num_epochs):
                print(f"\nEpoch {epoch + 1}/{self.num_epochs}")
                epoch_metrics = defaultdict(list)

                # Training loop with progress tracking
                with tqdm(total=len(loader), desc="Training") as pbar:
                    self.detection_system.model.train()

                    for batch_idx, (states, binary_labels, multi_labels) in enumerate(loader):
                        try:
                            # Convert numpy arrays to tensors if needed
                            if isinstance(states, np.ndarray):
                                states = torch.from_numpy(states).float()
                            if isinstance(binary_labels, np.ndarray):
                                binary_labels = torch.from_numpy(binary_labels).long()

                            # Move data to device in single operation
                            states = states.to(device, non_blocking=True)
                            binary_labels = binary_labels.to(device, non_blocking=True)

                            # Apply data augmentation if configured
                            if hasattr(self.detection_system, 'data_augmentation') and random.random() < 0.3:
                                states = self.detection_system.data_augmentation.augment(states)

                            # Forward pass with automatic mixed precision if enabled
                            with torch.cuda.amp.autocast() if self.config.use_amp else contextlib.nullcontext():
                                # Forward pass
                                self.detection_system.optimizer.zero_grad()
                                outputs = self.detection_system.model(states)

                                # Prepare DQN targets
                                q_values = outputs['q_values']
                                action_probs = outputs['action_probs']
                                state_value = outputs['state_value']

                                # Compute targets for both DQN and Actor-Critic
                                targets = {
                                    'q_targets': q_values.clone().detach(),
                                    'actions': binary_labels,
                                    'returns': state_value.clone().detach(),
                                    'labels': binary_labels
                                }

                                # Compute combined loss
                                loss_dict = self.criterion(outputs, targets)
                                total_loss = loss_dict['total_loss']

                                # Check for loss validity
                                if torch.isnan(total_loss) or torch.isinf(total_loss):
                                    print(f"Warning: Invalid loss value detected in batch {batch_idx}")
                                    continue

                            # Backward pass with gradient scaling if using AMP
                            if self.config.use_amp and hasattr(self.detection_system, 'scaler'):
                                self.detection_system.scaler.scale(total_loss).backward()
                                self.detection_system.scaler.step(self.detection_system.optimizer)
                                self.detection_system.scaler.update()
                            else:
                                total_loss.backward()
                                # Clip gradients for stability
                                torch.nn.utils.clip_grad_norm_(
                                    self.detection_system.model.parameters(), 
                                    self.config.gradient_clip
                                )

                                # Optimizer step with TPU/GPU handling
                                if use_tpu:
                                    xm.optimizer_step(self.detection_system.optimizer)
                                    xm.mark_step()
                                else:
                                    self.detection_system.optimizer.step()

                            # Update metrics
                            metrics = {
                                'loss': total_loss.item(),
                                'dqn_loss': loss_dict['dqn_loss'],
                                'policy_loss': loss_dict['policy_loss'],
                                'value_loss': loss_dict['value_loss'],
                                'detection_loss': loss_dict['detection_loss'],
                                'q_value_mean': q_values.mean().item(),
                                'q_value_std': q_values.std().item(),
                                'policy_entropy': -(action_probs * torch.log(action_probs + 1e-10)).sum(1).mean().item()
                            }

                            # Update epoch metrics
                            for k, v in metrics.items():
                                epoch_metrics[k].append(v)

                            # Update detection metrics
                            with torch.no_grad():
                                predictions = outputs['final_output'].argmax(dim=1)
                                accuracy = (predictions == binary_labels).float().mean().item()
                                epoch_metrics['accuracy'].append(accuracy)

                            # Update progress bar
                            avg_metrics = {k: np.mean(v) for k, v in epoch_metrics.items()}
                            pbar.set_postfix(avg_metrics)
                            pbar.update(1)

                            # Memory management
                            if self.memory_monitor.check_memory():
                                print("\nHigh memory usage detected")
                                if torch.cuda.is_available():
                                    torch.cuda.empty_cache()
                                break

                        except Exception as e:
                            print(f"Error in batch {batch_idx}: {str(e)}")
                            continue

                    # Epoch end processing
                    avg_metrics = {k: np.mean(v) for k, v in epoch_metrics.items()}
                    print(f"\nEpoch {epoch + 1} Summary:")
                    for k, v in avg_metrics.items():
                        print(f"- {k}: {v:.4f}")

                    # Validation check
                    if self.val_generator and (epoch + 1) % 5 == 0:
                        val_metrics = self._evaluate(epoch)

                        # Update best metrics
                        self._update_best_metrics(val_metrics, epoch)

                        # Early stopping check
                        if self.early_stopping(val_metrics.get('accuracy', 0), self.detection_system.model):
                            print("\nEarly stopping triggered")
                            break

                    # Save checkpoint if needed
                    if (epoch + 1) % 10 == 0:
                        self._save_checkpoint(f'epoch_{epoch + 1}')

                    # Cleanup
                    if use_tpu:
                        xm.mark_step()
                    else:
                        torch.cuda.empty_cache()

            print("\nTraining completed successfully")
            return True

        except Exception as e:
            print(f"Training error: {str(e)}")
            traceback.print_exc()
            return False


    
    def _train_batch(self, states, binary_labels, multi_labels):
        """Optimized training batch for TPU"""
        try:
            # Move data to TPU and preprocess in a single step
            states_tensor = xm.mesh_reduce('states', states, lambda x: torch.FloatTensor(x).to(xm.xla_device()))
            binary_labels_tensor = xm.mesh_reduce('labels', binary_labels, lambda x: torch.LongTensor(x).to(xm.xla_device()))

            # Forward pass optimization
            with torch.autocast(device_type=self.device.type):
                detection_output = self.detection_system.detect(states_tensor, labels=binary_labels_tensor)

                # Batch processing for replay buffer
                batch_experiences = list(zip(
                    states, 
                    binary_labels,
                    multi_labels,
                    detection_output['is_poisoning'],
                    states,
                    [True] * len(states)
                ))

                # Extend replay buffer efficiently
                self.detection_system.replay_buffer.extend(batch_experiences)

                # Train if enough samples
                if len(self.detection_system.replay_buffer) >= self.batch_size:
                    batch_metrics = self.detection_system.train(self.batch_size)
                    xm.mark_step()  # Explicit TPU sync point
                    return batch_metrics

            return None

        except Exception as e:
            print(f"Error in batch training: {str(e)}")
            return None
    
    

    def _update_best_metrics(self, val_metrics: Dict[str, float], epoch: int):
        """Update best metrics if current results are better"""
        try:
            for metric_name, value in val_metrics.items():
                if isinstance(value, (int, float)):
                    if metric_name not in self.best_metrics or value > self.best_metrics[metric_name]:
                        self.best_metrics[metric_name] = value
                        print(f"New best {metric_name}: {value:.4f}")
        except Exception as e:
            print(f"Warning: Error updating best metrics: {str(e)}")

    def _log_metrics(self, metrics: Dict[str, float]):
        """Log metrics to tensorboard and update history

        Args:
            metrics: Dictionary containing metric names and values
        """
        try:
            # Log to tensorboard
            for name, value in metrics.items():
                self.writer.add_scalar(
                    f'metrics/{name}',
                    value,
                    self._global_step
                )

                # Update history
                self.performance_history[name].append(value)

            # Update global step
            self._global_step += 1

        except Exception as e:
            print(f"Warning: Error logging metrics: {str(e)}")

    def _log_attack_metrics(self, attack_id: int, predictions: np.ndarray, labels: np.ndarray):
        """Log metrics for specific attack type"""
        try:
            attack_info = self.label_handler.get_attack_info(attack_id)
            # Convert predictions to class indices if needed
            if len(predictions.shape) > 1:
                pred_indices = predictions.argmax(axis=1)
            else:
                pred_indices = predictions

            metrics = {
                'precision': precision_score(labels, pred_indices, average='binary'),
                'recall': recall_score(labels, pred_indices, average='binary'),
                'f1': f1_score(labels, pred_indices, average='binary'),
                'accuracy': accuracy_score(labels, pred_indices)
            }

            # Log to tensorboard
            for name, value in metrics.items():
                self.writer.add_scalar(
                    f'attack_metrics/{attack_info["attack_name"]}/{name}',
                    value,
                    self._global_step
                )

                # Store in attack metrics
                self.attack_metrics[attack_id][name].append(value)

        except Exception as e:
            print(f"Warning: Error logging attack metrics: {str(e)}")

    def _log_batch_metrics(self, batch_metrics: Dict[str, float]):
        """Log batch-level training metrics"""
        try:
            # Add batch metrics to history
            for name, value in batch_metrics.items():
                if name == 'total_loss':
                    self.train_losses.append(value)
                self.writer.add_scalar(f'batch/{name}', value, self._global_step)

        except Exception as e:
            print(f"Warning: Error logging batch metrics: {str(e)}")

    def _log_epoch_metrics(self, epoch: int, metrics: Dict[str, float]):
        """Log epoch-level metrics"""
        try:
            # Compute epoch averages
            epoch_metrics = {}
            for name, values in metrics.items():
                if values:  # Check if list is not empty
                    epoch_metrics[f'epoch_{name}'] = np.mean(values)

            # Log to tensorboard
            for name, value in epoch_metrics.items():
                self.writer.add_scalar(f'epoch/{name}', value, epoch)

            return epoch_metrics

        except Exception as e:
            print(f"Warning: Error logging epoch metrics: {str(e)}")
            return {}
    
    
    def _evaluate(self, epoch: int) -> Dict[str, float]:
        """Enhanced evaluation with comprehensive metrics and proper device handling"""
        self.detection_system.model.eval()
        device = self.detection_system.device
        all_metrics = {}
        per_attack_metrics = defaultdict(list)

        try:
            with torch.no_grad():
                # Get validation data with proper handling
                eval_data, eval_binary_labels, eval_multi_labels = next(iter(self.val_generator))

                # Convert numpy arrays to tensors if needed
                if isinstance(eval_data, np.ndarray):
                    eval_data = torch.from_numpy(eval_data).float()
                if isinstance(eval_binary_labels, np.ndarray):
                    eval_binary_labels = torch.from_numpy(eval_binary_labels).long()
                if isinstance(eval_multi_labels, np.ndarray):
                    eval_multi_labels = torch.from_numpy(eval_multi_labels).long()

                # Move to device
                eval_data = eval_data.to(device)
                eval_binary_labels = eval_binary_labels.to(device)
                eval_multi_labels = eval_multi_labels.to(device)

                # Model prediction
                outputs = self.detection_system.model(eval_data)
                predictions = torch.argmax(outputs['final_output'], dim=1)

                # Calculate basic metrics
                accuracy = (predictions == eval_binary_labels).float().mean().item()
                predictions_np = predictions.cpu().numpy()
                labels_np = eval_binary_labels.cpu().numpy()

                metrics = {
                    'accuracy': accuracy,
                    'precision': precision_score(labels_np, predictions_np, average='binary'),
                    'recall': recall_score(labels_np, predictions_np, average='binary'),
                    'f1': f1_score(labels_np, predictions_np, average='binary'),
                    'avg_q_value': outputs['q_values'].mean().item(),
                    'policy_entropy': -(outputs['action_probs'] * 
                                      torch.log(outputs['action_probs'] + 1e-10)).sum(1).mean().item()
                }

                # Per-attack metrics
                if self.label_handler:
                    for attack_id in self.label_handler.attack_types:
                        attack_mask = eval_multi_labels == attack_id
                        if torch.any(attack_mask):
                            attack_preds = predictions[attack_mask]
                            attack_labels = eval_binary_labels[attack_mask]

                            attack_accuracy = (attack_preds == attack_labels).float().mean().item()
                            attack_name = self.label_handler.get_attack_info(attack_id)['attack_name']
                            per_attack_metrics[attack_name] = {
                                'accuracy': attack_accuracy,
                                'count': int(attack_mask.sum().item())
                            }

                # Uncertainty metrics
                detection_probs = F.softmax(outputs['final_output'], dim=1)
                entropy = -(detection_probs * torch.log(detection_probs + 1e-10)).sum(dim=1)
                metrics['prediction_entropy'] = entropy.mean().item()
                metrics['detection_confidence'] = detection_probs.max(dim=1)[0].mean().item()

                # Combine all metrics
                all_metrics.update(metrics)
                all_metrics['per_attack'] = dict(per_attack_metrics)

                # Log metrics
                self._log_evaluation_metrics(all_metrics, epoch)

                return all_metrics

        except Exception as e:
            print(f"Error during evaluation: {str(e)}")
            traceback.print_exc()
            return {'accuracy': 0.0, 'error': str(e)}


    def _calculate_metrics(self, model, data, labels) -> Dict[str, float]:
        """Calculate metrics with TPU optimization"""
        try:
            device = self.detection_system.device

            # Forward pass
            outputs = model.model(data)
            predictions = torch.argmax(outputs['final_output'], dim=1)

            # Calculate metrics on device
            accuracy = (predictions == labels).float().mean()

            # Move metrics to CPU for numpy operations
            if isinstance(device, torch_xla.xla_model.XlaDevice):
                accuracy = xm.mesh_reduce('accuracy', accuracy, lambda x: x.item())
                predictions = xm.mesh_reduce('predictions', predictions, lambda x: x.cpu())
                labels = xm.mesh_reduce('labels', labels, lambda x: x.cpu())
            else:
                accuracy = accuracy.item()
                predictions = predictions.cpu()
                labels = labels.cpu()

            # Calculate additional metrics
            metrics = {
                'accuracy': accuracy,
                'precision': precision_score(labels, predictions, average='binary'),
                'recall': recall_score(labels, predictions, average='binary'),
                'f1': f1_score(labels, predictions, average='binary')
            }

            return metrics

        except Exception as e:
            print(f"Error calculating metrics: {str(e)}")
            return {'accuracy': 0.0, 'error': str(e)}

    def _test_unknown_attacks(self, data) -> Dict[str, float]:
        """Test unknown attack detection with TPU support"""
        try:
            device = self.detection_system.device

            # Forward pass
            outputs = self.detection_system.model(data)
            detection_probs = F.softmax(outputs['final_output'], dim=1)

            # Calculate uncertainty metrics
            entropy = -(detection_probs * torch.log(detection_probs + 1e-10)).sum(dim=1)

            if isinstance(device, torch_xla.xla_model.XlaDevice):
                entropy = xm.mesh_reduce('entropy', entropy, lambda x: x.mean().item())
            else:
                entropy = entropy.mean().item()

            return {
                'unknown_detection_entropy': entropy,
                'unknown_detection_confidence': 1.0 - entropy
            }

        except Exception as e:
            print(f"Error in unknown attack detection: {str(e)}")
            return {
                'unknown_detection_entropy': 0.0,
                'unknown_detection_confidence': 0.0
            }



    def _log_per_attack_metrics(self, per_attack_metrics: Dict, epoch: int):
        """Log per-attack metrics to tensorboard"""
        for attack_name, metrics in per_attack_metrics.items():
            for metric_name, value in metrics.items():
                self.writer.add_scalar(
                    f'per_attack/{attack_name}/{metric_name}',
                    value,
                    epoch
                )

    def _save_checkpoint(self, identifier: str):
        """Save model checkpoint with additional metrics"""
        path = os.path.join(
            self.checkpoint_dir,
            f'checkpoint_{identifier}.pt'
        )

        checkpoint = {
            'epoch': self._epoch,
            'model_state_dict': self.detection_system.model.state_dict(),
            'optimizer_state_dict': self.detection_system.optimizer.state_dict(),
            'metrics': self.best_metrics,
            'global_step': self._global_step,
            'label_mapping': self.label_handler.label_mapping,  # Save label information
            'attack_types': self.label_handler.attack_types
        }

        torch.save(checkpoint, path)
        print(f"\nCheckpoint saved: {path}")

    def _handle_oom_error(self):
        """Handle out of memory error"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        self.batch_size = max(32, self.batch_size // 2)
        print(f"\nReducing batch size to: {self.batch_size}")

    def _check_memory(self):
        """Check memory usage"""
        if torch.cuda.is_available():
            memory_used = torch.cuda.memory_allocated()/1e9
            if memory_used > 0.9 * torch.cuda.get_device_properties(0).total_memory/1e9:
                print(f"\nHigh GPU memory usage ({memory_used:.2f}GB)")
                self._handle_oom_error()

    def _log_training_progress(self, epoch: int, batch_idx: int, metrics: Dict):
        """Log training progress"""
        try:
            # Log to tensorboard
            step = epoch * len(self.data_generator) + batch_idx
            for name, value in metrics.items():
                if isinstance(value, (int, float, np.number)):
                    self.writer.add_scalar(f'training/{name}', value, step)

            # Update global step
            self._global_step = step

        except Exception as e:
            print(f"Warning: Error logging progress: {str(e)}")

    def _handle_epoch_completion(self, epoch: int, epoch_metrics: Dict):
        """Handle end of epoch procedures"""
        try:
            # Calculate average metrics
            avg_metrics = {}
            for k, v in epoch_metrics.items():
                if isinstance(v, (list, np.ndarray)) and len(v) > 0:
                    if all(isinstance(x, (int, float, np.number)) for x in v):
                        avg_metrics[k] = float(np.mean(v))

            # Print epoch summary
            print(f"\nEpoch {epoch + 1} Complete:")
            print("-" * 50)
            print("Training Metrics:")
            for name, value in avg_metrics.items():
                print(f"- {name}: {value:.4f}")

            # Log to tensorboard
            for name, value in avg_metrics.items():
                self.writer.add_scalar(f'epoch/{name}', value, epoch)

            # Validation if available
            if self.val_generator and (epoch + 1) % 5 == 0:
                print("\nRunning Validation...")
                val_metrics = self._evaluate(epoch)
                self._update_best_metrics(val_metrics, epoch)
            
            # Self_progress_monitor
            self.progress_monitor.update(epoch, avg_metrics)
            if (epoch + 1) % 5 == 0:  # Plot every 5 epochs
                self.progress_monitor.plot_training_progress()
            
            # Memory management
            if self.memory_monitor.check_memory():
                self._handle_oom_error()

            print("\nCurrent Best Metrics:")
            for metric, value in self.best_metrics.items():
                if isinstance(value, (int, float)):
                    print(f"- Best {metric}: {value:.4f}")

        except Exception as e:
            print(f"Warning: Error in epoch completion handling: {str(e)}")
            traceback.print_exc()


    def _check_convergence(self, current_loss: float, last_loss: float,
                          plateau_counter: int, min_loss_change: float) -> Tuple[int, bool]:
        """Check if training has converged"""
        if abs(current_loss - last_loss) < min_loss_change:
            plateau_counter += 1
        else:
            plateau_counter = 0

        return plateau_counter, plateau_counter >= self.config.patience

    def _should_stop_training(self, iteration_counter: int, max_iterations: int,
                            avg_loss: float, plateau_counter: int) -> bool:
        """Check if training should stop"""
        if iteration_counter >= max_iterations:
            print(f"\nReached maximum iterations ({max_iterations})")
            return True

        if avg_loss < 1e-4:
            print(f"\nReached minimum loss threshold")
            return True

        if plateau_counter >= self.config.patience:
            print(f"\nLoss plateau reached")
            return True

        return False
    
    def _log_evaluation_metrics(self, metrics: Dict, epoch: int):
        """Log evaluation metrics properly"""
        for metric_name, value in metrics.items():
            if isinstance(value, (int, float)):
                self.writer.add_scalar(f'eval/{metric_name}', value, epoch)
                print(f"{metric_name}: {value:.4f}")

## Dataset Processor

In [ ]:
# Initialize
config = ModelConfig()  # Will automatically detect and setup TPU

class DatasetProcessor:
    """Handles dataset processing and model training"""
    def __init__(self, config: ModelConfig):
        self.config = config
        self.all_models = {}
        self.all_metrics = {}

    def process_single_dataset(self, dataset_type: str, data_path: str):
        """Process a single dataset"""
        try:
            # Load dataset using EnhancedDatasetLoader
            loader = EnhancedDatasetLoader(dataset_type, self.config)
            X, binary_labels, multi_labels, additional_info = loader.load_and_process_dataset(data_path)
            print(f"\nDataset loaded - Shape: {X.shape}")

            # Initialize components
            metrics_tracker = PoisoningDetectionMetrics(loader.label_handler)
            print("Created metrics tracker")

            # Create train and validation generators
            split_idx = int(0.8 * len(X))
            X_train, X_val = X[:split_idx], X[split_idx:]
            binary_train, binary_val = binary_labels[:split_idx], binary_labels[split_idx:]
            multi_train, multi_val = multi_labels[:split_idx], multi_labels[split_idx:]

            train_generator = MultiDatasetGenerator(
                {dataset_type: (X_train, binary_train, multi_train)},
                batch_size=self.config.batch_size,
                shuffle=True,
                num_workers=self.config.num_workers
            )
            print("Created data generator")

            val_generator = MultiDatasetGenerator(
                {dataset_type: (X_val, binary_val, multi_val)},
                batch_size=self.config.batch_size,
                shuffle=False,
                num_workers=self.config.num_workers
            )
            print("Created validation generator")

            # Initialize detection system
            detection_system = EnhancedPoisoningDetectionSystem(
                input_dim=X.shape[1],
                config=self.config,
                label_handler=loader.label_handler
            )
            print("Initialized detection system")

            # Setup training pipeline
            pipeline = ComprehensiveTrainingPipeline(
                detection_system=detection_system,
                data_generator=train_generator,
                val_generator=val_generator,
                config=self.config,
                label_handler=loader.label_handler
            )

            # Train
            print("\nStarting training...")
            pipeline.train()

            # Save additional info from enhanced processing
            self._save_additional_info(dataset_type, additional_info)

            return detection_system, metrics_tracker

        except Exception as e:
            print(f"Error processing dataset: {str(e)}")
            traceback.print_exc()
            return None, None

    def _save_additional_info(self, dataset_type: str, additional_info: Dict):
        """Save additional processing information"""
        save_path = os.path.join(
            self.config.checkpoint_dir,
            f'{dataset_type}_additional_info.pt'
        )
        torch.save(additional_info, save_path)
        print(f"\nSaved additional processing info to {save_path}")



## Results Evaluation

In [ ]:
class ResultsAnalyzer:
    """Analyze and format detection results per dataset"""
    def __init__(self):
        self.metrics = {
            'cic': defaultdict(dict),
            'ton': defaultdict(dict),
            'cse': defaultdict(dict)
        }
        
    def compute_per_dataset_metrics(self, dataset_type: str, predictions: np.ndarray, 
                                  true_labels: np.ndarray, attack_types: Dict[int, str]):
        """Compute comprehensive metrics for each attack type"""
        results = {}
        
        # Per-attack metrics
        for attack_id, attack_name in attack_types.items():
            mask = true_labels == attack_id
            if np.any(mask):
                results[attack_name] = {
                    'accuracy': accuracy_score(true_labels[mask], predictions[mask]),
                    'precision': precision_score(true_labels[mask], predictions[mask], average='weighted'),
                    'recall': recall_score(true_labels[mask], predictions[mask], average='weighted'),
                    'f1': f1_score(true_labels[mask], predictions[mask], average='weighted'),
                    'samples': int(np.sum(mask))
                }
        
        # Overall metrics
        results['overall'] = {
            'accuracy': accuracy_score(true_labels, predictions),
            'macro_precision': precision_score(true_labels, predictions, average='macro'),
            'macro_recall': recall_score(true_labels, predictions, average='macro'),
            'macro_f1': f1_score(true_labels, predictions, average='macro'),
            'weighted_f1': f1_score(true_labels, predictions, average='weighted')
        }
        
        self.metrics[dataset_type] = results
        return results

    def generate_results_table(self) -> pd.DataFrame:
        """Generate tabular results for all datasets"""
        all_results = []
        
        for dataset in ['cic', 'ton', 'cse']:
            dataset_metrics = self.metrics[dataset]
            
            for attack, metrics in dataset_metrics.items():
                if attack != 'overall':
                    row = {
                        'Dataset': dataset.upper(),
                        'Attack Type': attack,
                        'Accuracy': metrics.get('accuracy', 0) * 100,
                        'Precision': metrics.get('precision', 0) * 100,
                        'Recall': metrics.get('recall', 0) * 100,
                        'F1-Score': metrics.get('f1', 0) * 100,
                        'Samples': metrics.get('samples', 0)
                    }
                    all_results.append(row)
                    
        return pd.DataFrame(all_results)

    def print_summary(self):
        """Print comprehensive summary of results"""
        print("\nDetection Results Summary")
        print("=" * 80)
        
        for dataset in ['cic', 'ton', 'cse']:
            print(f"\n{dataset.upper()} Dataset Results:")
            print("-" * 40)
            
            if dataset in self.metrics and 'overall' in self.metrics[dataset]:
                overall = self.metrics[dataset]['overall']
                print(f"Overall Accuracy: {overall['accuracy']*100:.2f}%")
                print(f"Macro F1-Score: {overall['macro_f1']*100:.2f}%")
                print(f"Weighted F1-Score: {overall['weighted_f1']*100:.2f}%")
                
                print("\nPer-Attack Performance:")
                for attack, metrics in self.metrics[dataset].items():
                    if attack != 'overall':
                        print(f"\n{attack}:")
                        print(f"- Accuracy: {metrics['accuracy']*100:.2f}%")
                        print(f"- F1-Score: {metrics['f1']*100:.2f}%")
                        print(f"- Samples: {metrics['samples']}")
            else:
                print("No results available")
    

## Progress Monitor

In [ ]:
class ProgressMonitor:
    def __init__(self, save_dir='visualizations'):
        self.metrics_history = defaultdict(list)
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        
    def update(self, epoch, metrics):
        for k, v in metrics.items():
            self.metrics_history[k].append(v)
            
    def plot_training_progress(self):
        plt.figure(figsize=(15, 5))
        for metric in ['accuracy', 'loss', 'f1']:
            if metric in self.metrics_history:
                plt.plot(self.metrics_history[metric], label=metric)
        plt.legend()
        plt.title('Training Progress')
        plt.savefig(f'{self.save_dir}/training_progress.png')
        plt.close()
        

## Results Visualization

In [ ]:
class ResultsVisualizer:
    """Visualization tools for detection results"""
    def __init__(self, save_dir='visualizations'):
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        # Use a built-in matplotlib style instead of 'seaborn'
        plt.style.use('seaborn-v0_8-darkgrid')  # or simply use 'default'
        
        # Set default plot parameters
        plt.rcParams['figure.figsize'] = (12, 8)
        plt.rcParams['axes.grid'] = True
        plt.rcParams['font.size'] = 10
        
    def plot_confusion_matrices(self, results, dataset_type):
        """Plot confusion matrices for each attack type"""
        try:
            plt.figure(figsize=(15, 5))
            if dataset_type in results and 'metrics' in results[dataset_type]:
                metrics = results[dataset_type]['metrics']
                if hasattr(metrics, 'confusion_matrix'):
                    sns.heatmap(metrics.confusion_matrix, annot=True, fmt='d',
                               xticklabels=['Normal', 'Attack'],
                               yticklabels=['Normal', 'Attack'],
                               cmap='Blues')  # Specify colormap explicitly
                    plt.title(f'{dataset_type.upper()} Confusion Matrix')
                    plt.tight_layout()
                    plt.savefig(f'{self.save_dir}/{dataset_type}_confusion_matrix.png')
                    plt.close()
        except Exception as e:
            print(f"Error plotting confusion matrix: {str(e)}")
        
    def plot_roc_curves(self, results, dataset_type):
        """Plot ROC curves for each dataset"""
        try:
            plt.figure(figsize=(10, 6))
            if dataset_type in results and 'metrics' in results[dataset_type]:
                metrics = results[dataset_type]['metrics']
                if hasattr(metrics, 'fpr') and hasattr(metrics, 'tpr'):
                    plt.plot(metrics.fpr, metrics.tpr, 
                            label=f'{dataset_type} (AUC = {metrics.roc_auc:.2f})')
                    plt.plot([0, 1], [0, 1], 'k--')  # Add diagonal line
                    plt.xlabel('False Positive Rate')
                    plt.ylabel('True Positive Rate')
                    plt.title(f'{dataset_type.upper()} ROC Curve')
                    plt.legend()
                    plt.grid(True)
                    plt.savefig(f'{self.save_dir}/{dataset_type}_roc_curve.png')
                    plt.close()
        except Exception as e:
            print(f"Error plotting ROC curve: {str(e)}")

    def plot_attack_distribution(self, results, dataset_type):
        """Plot distribution of attacks for each dataset"""
        try:
            if dataset_type in results and 'metrics' in results[dataset_type]:
                metrics = results[dataset_type]['metrics']
                if hasattr(metrics, 'true_labels'):
                    plt.figure(figsize=(15, 8))
                    unique_labels, counts = np.unique(metrics.true_labels, return_counts=True)
                    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))
                    plt.bar(unique_labels, counts, color=colors)
                    plt.title(f'Attack Distribution - {dataset_type.upper()}')
                    plt.xlabel('Attack Types')
                    plt.ylabel('Number of Samples')
                    plt.xticks(rotation=45)
                    plt.grid(True, axis='y')
                    plt.tight_layout()
                    plt.savefig(f'{self.save_dir}/{dataset_type}_attack_distribution.png')
                    plt.close()
        except Exception as e:
            print(f"Error plotting attack distribution: {str(e)}")

    def plot_performance_comparison(self, results):
        """Plot performance comparison across datasets"""
        try:
            plt.figure(figsize=(12, 6))
            metrics_data = []
            for dataset_type, result in results.items():
                if 'metrics' in result:
                    metrics = result['metrics']
                    if hasattr(metrics, 'accuracy'):
                        metrics_data.append({
                            'Dataset': dataset_type.upper(),
                            'Accuracy': metrics.accuracy,
                            'Precision': metrics.precision,
                            'Recall': metrics.recall,
                            'F1-Score': metrics.f1,
                            'Detection Rate': metrics.detection_rate,
                            'False Alarm Rate': metrics.false_alarm_rate
                        })
            
            if metrics_data:
                df = pd.DataFrame(metrics_data)
                ax = df.set_index('Dataset').plot(kind='bar', 
                                                width=0.8,
                                                colormap='viridis')
                plt.title('Performance Metrics Comparison')
                plt.ylabel('Score')
                plt.xlabel('Dataset')
                plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                plt.grid(True, axis='y')
                plt.tight_layout()
                plt.savefig(f'{self.save_dir}/performance_comparison.png',
                          bbox_inches='tight')
                plt.close()
        except Exception as e:
            print(f"Error plotting performance comparison: {str(e)}")
    



## Main Execution 1

In [ ]:
def tpu_training_wrapper(func):
    """Wrapper to handle TPU execution modes with proper device management"""
    def wrapper(*args, **kwargs):
        try:
            if xm.xrt_world_size() > 1:
                # Multi-core TPU execution
                def _mp_fn(index, *args):
                    device = xm.xla_device()
                    results = func(*args)
                    xm.mark_step()
                    return results
                
                return xmp.spawn(_mp_fn, args=args, nprocs=8)
            else:
                # Single core execution
                device = xm.xla_device()
                results = func(*args)
                xm.mark_step()
                return results
        except Exception as e:
            print(f"Error in TPU execution: {str(e)}")
            traceback.print_exc()
            return None
    return wrapper

# Add TPU device check helper at the top
def is_tpu_device(device) -> bool:
    """Safely check if device is TPU with enhanced error handling"""
    try:
        import torch_xla
        import torch_xla.core.xla_model as xm
        if isinstance(device, str):
            return device.startswith('xla')
        return hasattr(device, 'type') and device.type.startswith('xla')
    except (ImportError, AttributeError) as e:
        print(f"TPU check failed: {str(e)}")
        return False
    except Exception as e:
        print(f"Unexpected error in TPU check: {str(e)}")
        return False

def main_single_dataset():
    """Main function to process datasets one at a time with TPU support"""
    try:
        # Setup with TPU awareness and metrics
        import torch_xla.core.xla_model as xm
        import torch_xla.debug.metrics as met
        
        device = xm.xla_device()  # Get TPU device directly
        config = ModelConfig()
        
        if not setup_system():
            raise RuntimeError("System setup failed")

        # Dataset configurations
        datasets = {
            'cic': {'path': '/kaggle/input/poisoning-i/CIC_IoT_M3.csv', 'description': 'CIC-IDS Dataset'},
            'ton': {'path': '/kaggle/input/poisoning-i/UNSW_TON_IoT.csv', 'description': 'TON-IoT Dataset'},
            'cse': {'path': '/kaggle/input/poisoning-i/CSE-CIC_2018.csv', 'description': 'CSE-CIC Dataset'}
        }

        # Initialize analyzers and visualizers
        results_analyzer = ResultsAnalyzer()
        attack_mapper = AttackTypeMapper()
        enhanced_metrics = EnhancedMetrics()
        visualizer = ResultsVisualizer()
        
        # Process one dataset at a time with TPU handling
        results = {}
        for dataset_type, info in datasets.items():
            print(f"\n{'='*50}")
            print(f"Processing {dataset_type.upper()} Dataset")
            print(f"{'='*50}")
            
            try:
                trainer = SingleDatasetTrainer(config, dataset_type)
                model, metrics = trainer.train_on_dataset(info['path'])
                
                results[dataset_type] = {
                    'model': model,
                    'metrics': metrics
                }

                # Generate metrics and visualizations for this dataset
                attack_mapping = AttackTypeMapper.get_mappings()[dataset_type]
                
                # Compute comprehensive metrics
                per_attack_results = enhanced_metrics.per_attack_metrics(
                    metrics.true_labels,
                    metrics.predictions,
                    attack_mapping
                )
                
                # Generate visualizations
                visualizer.plot_confusion_matrices(results, dataset_type)
                visualizer.plot_roc_curves(results, dataset_type)
                visualizer.plot_attack_distribution(results, dataset_type)
                
                # Compute per-dataset metrics
                dataset_results = results_analyzer.compute_per_dataset_metrics(
                    dataset_type,
                    metrics.predictions,
                    metrics.true_labels,
                    trainer.loader.label_handler.attack_types
                )
                
                print(f"\nCompleted processing {dataset_type.upper()}")
                print("Cleaning up memory...")
                gc.collect()
                xm.mark_step()  # Ensure TPU operations are completed
                
            except Exception as e:
                print(f"Error processing {dataset_type} dataset: {str(e)}")
                traceback.print_exc()
                continue

        # Generate and display overall results
        results_table = results_analyzer.generate_results_table()
        print("\nDetailed Results Table:")
        print(results_table.to_string())
        results_analyzer.print_summary()
        
        # Generate overall performance comparison
        visualizer.plot_performance_comparison(results)

        return results

    except Exception as e:
        print(f"Error in main execution: {str(e)}")
        traceback.print_exc()
        return None
    
    # Generate and display results
    results_table = results_analyzer.generate_results_table()
    print("\nDetailed Results Table:")
    print(results_table.to_string())
    results_analyzer.print_summary() 
    
    attack_mapper = AttackTypeMapper()
    enhanced_metrics = EnhancedMetrics()
    visualizer = ResultsVisualizer()
    
    # In main_single_dataset(), after processing results:
    for dataset_type, result in results.items():
        attack_mapping = AttackTypeMapper.get_mappings()[dataset_type]
        per_attack_results = enhanced_metrics.per_attack_metrics(
            result['metrics'].true_labels,
            result['metrics'].predictions,
            attack_mapping
        )
        visualizer.plot_attack_distribution(results, dataset_type)
    

if __name__ == "__main__":
    try:
        # First ensure TPU is available
        print("Checking TPU availability...")
        import torch_xla
        import torch_xla.core.xla_model as xm
        import torch_xla.debug.metrics as met
        
        if torch_xla is None:
            raise ImportError("torch_xla not available")
        
        print(f"TPU devices found: {xm.xrt_world_size()}")
        
        # Run with TPU support
        results = main_single_dataset()
        
        # Print training summary
        if results:
            print("\nTraining Summary:")
            for dataset_type, result in results.items():
                print(f"\n{dataset_type.upper()} Results:")
                if result['metrics']:
                    for metric_name, value in result['metrics'].items():
                        if isinstance(value, (int, float)):
                            print(f"- {metric_name}: {value:.4f}")
        
        # Print TPU metrics safely
        try:
            if xm.xrt_world_size() > 0:
                print("\nTPU Metrics:")
                print(met.metrics_report())
        except Exception as e:
            print(f"Note: Could not generate TPU metrics report: {str(e)}")
            
    except Exception as e:
        print(f"Failed to initialize training: {str(e)}")
        traceback.print_exc()
        




---

